# 第九章 模組、程式包與匯入

本章學習 Go 的模組系統，包括模組、程式包的概念、go.mod 的使用、程式包的建立與管理、第三方套件的使用和版本控制。這是構建大型 Go 應用程式的重要基礎。

## 版本庫、模組與程式包

在 Go 的生態系統中，有三個重要概念需要理解：版本庫（repository）、模組（module）和程式包（package）。

- **版本庫**：存放代碼的地方，如 GitHub repository
- **模組**：一個或多個相關程式包的集合，有自己的 go.mod 檔案
- **程式包**：一個目錄內的 Go 檔案集合，共享相同的 package 宣告

模組是 Go 1.11 引入的依賴管理系統，取代了過去的 GOPATH 工作區模式。每個模組都有唯一的模組路徑，通常與版本庫的 URL 對應。

In [ ]:
/* 展示版本庫、模組與程式包的層級關係 */
package main

import (
    "fmt"
    // 模組路徑/程式包名稱
    "github.com/example/myproject/utils"
    "github.com/example/myproject/handlers"
)

func main() {
    // 使用不同程式包的功能
    result := utils.Calculate(10, 20) // 輸出: 30
    fmt.Println("計算結果:", result)
    
    handlers.HandleRequest("GET", "/users") // 輸出: 處理 GET /users 請求
}

## go.mod

go.mod 檔案是 Go 模組的核心，定義了模組的身份和依賴關係。它包含：

- **module 指令**：定義模組路徑
- **go 指令**：指定 Go 版本
- **require 指令**：列出依賴的模組和版本
- **replace 指令**：替換依賴的模組路徑
- **exclude 指令**：排除特定版本的模組

go.mod 檔案會自動管理，但也可以手動編輯。使用 `go mod` 命令可以操作模組。

In [ ]:
/* 創建和管理 go.mod 檔案 */
package main

import "fmt"

// 1. 初始化模組
// go mod init github.com/example/myproject

// go.mod 檔案內容示例:
/*
module github.com/example/myproject

go 1.19

require (
    github.com/gin-gonic/gin v1.9.1
    gorm.io/gorm v1.25.1
)

require (
    // 間接依賴會自動添加
    github.com/json-iterator/go v1.1.12 // indirect
)
*/

func main() {
    fmt.Println("模組已初始化")
    
    // 常用 go mod 命令：
    // go mod init <module-name>    // 初始化模組
    // go mod tidy                 // 整理依賴
    // go mod download             // 下載依賴
    // go mod vendor               // 建立 vendor 目錄
    // go mod graph                // 顯示依賴圖
}

## 建構程式包

程式包是 Go 代碼組織的基本單位。每個程式包由同一目錄下的一個或多個 Go 檔案組成，這些檔案都有相同的 package 宣告。

程式包的主要特性：
- **封裝性**：控制哪些識別符號可以被外部存取
- **命名空間**：提供識別符號的命名空間
- **重用性**：可以被其他程式包匯入使用
- **測試性**：每個程式包都可以有對應的測試

程式包名稱應該簡短、有意義，通常與目錄名相同。

In [ ]:
/* 程式包的基本結構示例 */
// 檔案: myproject/mathutil/calculator.go
package mathutil // 程式包宣告

import "fmt"

// 首字母大寫的識別符號是公開的，可以被外部存取
func Add(a, b int) int {
    return add(a, b)
}

// 首字母小寫的識別符號是私有的，只能在同一程式包內使用
func add(a, b int) int {
    fmt.Println("執行加法運算") // 輸出: 執行加法運算
    return a + b
}

// 公開的常數
const MaxValue = 1000

// 私有的變數
var counter int

### 匯入與匯出

Go 的匯入與匯出機制非常簡單：

**匯出規則**：
- 首字母大寫的識別符號（變數、函式、類型、常數）是公開的
- 首字母小寫的識別符號是私有的，只能在同一程式包內使用

**匯入方式**：
- 標準匯入：`import "package"`
- 別名匯入：`import alias "package"`
- 點匯入：`import . "package"`（不推薦）
- 空白匯入：`import _ "package"`（執行 init 函式）

In [ ]:
/* 匯入與匯出的詳細範例 */
package main

import (
    "fmt"
    "math" // 標準匯入
    m "math/rand" // 別名匯入
    _ "image/png" // 空白匯入（註冊 PNG 解碼器）
)

// 程式包級別的公開函式
func PublicFunction() string {
    return "這是公開函式"
}

// 程式包級別的私有函式
func privateFunction() string {
    return "這是私有函式"
}

// 公開結構體
type User struct {
    Name string // 公開欄位
    age  int    // 私有欄位
}

// 私有結構體
type session struct {
    token string
}

func main() {
    // 使用標準匯入
    result := math.Sqrt(16) // 輸出: 4
    fmt.Println("平方根:", result)
    
    // 使用別名匯入
    num := m.Intn(100) // 輸出: 隨機數 0-99
    fmt.Println("隨機數:", num)
    
    // 使用自定義結構體
    user := User{Name: "Alice"} // 無法直接設定 age，因為它是私有的
    fmt.Println("用戶:", user.Name)
}

### 建立與使用程式包

建立自定義程式包的步驟：

1. 創建目錄結構
2. 在每個目錄中創建 Go 檔案
3. 使用 package 宣告指定程式包名稱
4. 定義公開和私有的識別符號
5. 在其他程式包中匯入使用

程式包的目錄結構建議：
```
myproject/
├── go.mod
├── main.go
├── handlers/
│   └── user.go
├── models/
│   └── user.go
└── utils/
    └── helper.go
```

In [ ]:
/* 建立完整的程式包示例 */

// 檔案: models/user.go
package models

import "time"

type User struct {
    ID       int       `json:"id"`
    Name     string    `json:"name"`
    Email    string    `json:"email"`
    CreateAt time.Time `json:"created_at"`
}

func NewUser(name, email string) *User {
    return &User{
        Name:     name,
        Email:    email,
        CreateAt: time.Now(),
    }
}

func (u *User) GetDisplayName() string {
    if u.Name != "" {
        return u.Name
    }
    return u.Email
}

// 檔案: utils/validator.go
package utils

import "regexp"

var emailRegex = regexp.MustCompile(`^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$`)

func ValidateEmail(email string) bool {
    return emailRegex.MatchString(email)
}

func IsEmpty(str string) bool {
    return len(str) == 0
}

// 檔案: main.go - 使用自定義程式包
package main

import (
    "fmt"
    "myproject/models"  // 匯入自定義程式包
    "myproject/utils"   // 匯入工具程式包
)

func main() {
    // 使用 models 程式包
    user := models.NewUser("Alice", "alice@example.com")
    fmt.Println("用戶:", user.GetDisplayName()) // 輸出: 用戶: Alice
    
    // 使用 utils 程式包
    if utils.ValidateEmail(user.Email) {
        fmt.Println("郵箱格式正確") // 輸出: 郵箱格式正確
    }
    
    if utils.IsEmpty(user.Name) {
        fmt.Println("名稱為空")
    } else {
        fmt.Println("名稱:", user.Name) // 輸出: 名稱: Alice
    }
}

### 為程式包命名

程式包命名是 Go 程式設計中的重要考量，好的命名可以讓代碼更易理解和使用。

**命名原則**：
- **簡短明確**：使用簡短但有意義的名稱
- **小寫字母**：程式包名稱應該全部小寫，不使用底線或混合大小寫
- **避免複數**：通常使用單數形式（如 `user` 而非 `users`）
- **避免通用名稱**：避免使用 `util`、`common`、`base` 等過於通用的名稱
- **符合功能**：名稱應該反映程式包的核心功能

**常見的好範例**：
- `http` - 處理 HTTP 協議
- `json` - JSON 編碼解碼
- `time` - 時間處理
- `log` - 日誌功能
- `crypto` - 密碼學功能

In [ ]:
/* 程式包命名的最佳實踐 */
package main

import (
    "fmt"
    "net/http" // 好的命名：簡短、清楚
    "encoding/json" // 好的命名：描述功能
    "database/sql" // 好的命名：分層結構
)

// 好的程式包名稱範例：

// 檔案: user/service.go - 專注於用戶相關功能
package user

type Service struct {
    db Database
}

func (s *Service) Create(name, email string) error {
    fmt.Println("創建用戶:", name) // 輸出: 創建用戶: Alice
    return nil
}

// 檔案: auth/token.go - 專注於認證功能
package auth

type TokenManager struct {
    secret string
}

func (tm *TokenManager) Generate(userID int) string {
    fmt.Printf("為用戶 %d 生成令牌\n", userID) // 輸出: 為用戶 123 生成令牌
    return "token123"
}

// 檔案: logger/logger.go - 專注於日誌功能
package logger

type Logger struct {
    level string
}

func (l *Logger) Info(message string) {
    fmt.Println("[INFO]", message) // 輸出: [INFO] 系統啟動
}

func main() {
    // 避免的不良命名範例：
    // package util      // 太通用
    // package helpers   // 太模糊
    // package myStuff   // 混合大小寫
    // package user_mgmt // 使用底線
    
    fmt.Println("程式包命名示例")
    
    // 好的程式包使用起來語意清楚
    userService := &user.Service{}
    userService.Create("Alice", "alice@example.com")
    
    tokenMgr := &auth.TokenManager{}
    token := tokenMgr.Generate(123)
    fmt.Println("生成的令牌:", token) // 輸出: 生成的令牌: token123
    
    log := &logger.Logger{}
    log.Info("系統啟動")
}

### 如何組織你的模組

模組的組織結構對於維護性和可讀性至關重要。好的組織結構能夠清楚地表達代碼的意圖和責任劃分。

**組織原則**：
1. **按功能域分組**：將相關的功能放在同一個程式包中
2. **分層架構**：區分業務邏輯、數據訪問、API 處理等層次
3. **依賴方向**：高層模組不應該依賴低層模組的具體實現
4. **單一責任**：每個程式包應該有明確的單一責任

**常見的目錄結構模式**：
```
myproject/
├── cmd/           # 應用程式入口點
├── internal/      # 內部程式包（不對外暴露）
├── pkg/           # 可重用的程式包
├── api/           # API 定義
├── web/           # Web 資源
└── docs/          # 文檔
```

**分層架構範例**：
- **Handler 層**：處理 HTTP 請求
- **Service 層**：業務邏輯處理
- **Repository 層**：數據訪問
- **Model 層**：數據模型定義

In [ ]:
/* 模組組織的策略 */
package main

import "fmt"

// 1. 按功能域組織 - 電商系統範例
/*
ecommerce/
├── go.mod
├── cmd/
│   └── server/
│       └── main.go          # 應用程式入口
├── internal/                # 內部程式包
│   ├── user/               # 用戶功能域
│   │   ├── handler.go      # HTTP 處理器
│   │   ├── service.go      # 業務邏輯
│   │   └── repository.go   # 數據訪問
│   ├── product/            # 商品功能域
│   │   ├── handler.go
│   │   ├── service.go
│   │   └── repository.go
│   └── order/              # 訂單功能域
│       ├── handler.go
│       ├── service.go
│       └── repository.go
└── pkg/                    # 可重用程式包
    ├── database/           # 數據庫工具
    ├── logger/            # 日誌工具
    └── validator/         # 驗證工具
*/

// 2. 分層架構示例

// 數據模型層
type User struct {
    ID    int    `json:"id"`
    Name  string `json:"name"`
    Email string `json:"email"`
}

// Repository 層 - 數據訪問
type UserRepository interface {
    Create(user *User) error
    GetByID(id int) (*User, error)
}

type userRepository struct {
    db Database
}

func (r *userRepository) Create(user *User) error {
    fmt.Println("在數據庫中創建用戶:", user.Name) // 輸出: 在數據庫中創建用戶: Alice
    return nil
}

func (r *userRepository) GetByID(id int) (*User, error) {
    fmt.Println("從數據庫獲取用戶 ID:", id) // 輸出: 從數據庫獲取用戶 ID: 1
    return &User{ID: id, Name: "Alice", Email: "alice@example.com"}, nil
}

// Service 層 - 業務邏輯
type UserService struct {
    repo UserRepository
}

func (s *UserService) RegisterUser(name, email string) (*User, error) {
    fmt.Println("註冊新用戶:", name) // 輸出: 註冊新用戶: Alice
    
    user := &User{Name: name, Email: email}
    if err := s.repo.Create(user); err != nil {
        return nil, err
    }
    
    fmt.Println("用戶註冊成功") // 輸出: 用戶註冊成功
    return user, nil
}

// Handler 層 - HTTP 處理
type UserHandler struct {
    service *UserService
}

func (h *UserHandler) CreateUser(name, email string) {
    fmt.Println("處理創建用戶請求") // 輸出: 處理創建用戶請求
    
    user, err := h.service.RegisterUser(name, email)
    if err != nil {
        fmt.Println("創建用戶失敗:", err)
        return
    }
    
    fmt.Println("返回用戶數據:", user.Name) // 輸出: 返回用戶數據: Alice
}

func main() {
    fmt.Println("模組組織架構示例")
    
    // 依賴注入 - 從底層到高層
    repo := &userRepository{}
    service := &UserService{repo: repo}
    handler := &UserHandler{service: service}
    
    // 模擬 HTTP 請求處理
    handler.CreateUser("Alice", "alice@example.com")
    
    fmt.Println("\n分層架構的優點:")
    fmt.Println("- 職責清楚分離") // 輸出: - 職責清楚分離
    fmt.Println("- 易於測試和維護") // 輸出: - 易於測試和維護
    fmt.Println("- 支持依賴注入") // 輸出: - 支持依賴注入
}

### 覆寫程式包的名稱

當匯入的程式包名稱發生衝突，或者程式包名稱過長、不夠直觀時，可以使用別名來覆寫程式包的名稱。

**使用別名的情況**：
1. **名稱衝突**：兩個程式包有相同的名稱
2. **名稱過長**：簡化長程式包名稱
3. **提高可讀性**：使用更直觀的別名
4. **版本區別**：區分同一程式包的不同版本

**別名匯入語法**：
```go
import alias "package/path"
```

**特殊匯入方式**：
- **點匯入**：`import . "package"` - 將程式包的識別符號直接匯入當前命名空間
- **空白匯入**：`import _ "package"` - 只執行程式包的 init 函式
- **別名匯入**：`import alias "package"` - 使用自定義別名

In [ ]:
/* 程式包別名的使用 */
package main

import (
    "fmt"
    "math/rand" // 標準匯入
    
    // 別名匯入 - 解決名稱衝突
    cryptorand "crypto/rand" // 加密隨機數
    mathrand "math/rand"     // 數學隨機數
    
    // 簡化長程式包名稱
    uuid "github.com/google/uuid"
    pb "google.golang.org/protobuf/proto"
    
    // 版本別名
    v1 "myproject/api/v1"
    v2 "myproject/api/v2"
    
    // 點匯入 (不推薦，會污染命名空間)
    . "strings"
    
    // 空白匯入 (只執行 init 函式)
    _ "image/jpeg" // 註冊 JPEG 解碼器
    _ "image/png"  // 註冊 PNG 解碼器
)

func main() {
    fmt.Println("程式包別名使用示例")
    
    // 1. 解決名稱衝突
    fmt.Println("\n1. 名稱衝突解決:")
    
    // 使用數學隨機數
    mathrand.Seed(42)
    num1 := mathrand.Intn(100)
    fmt.Println("數學隨機數:", num1) // 輸出: 數學隨機數: 28
    
    // 使用加密隨機數
    buf := make([]byte, 4)
    cryptorand.Read(buf)
    fmt.Printf("加密隨機數: %x\n", buf) // 輸出: 加密隨機數: [隨機16進制]
    
    // 2. 簡化長程式包名稱
    fmt.Println("\n2. 簡化長程式包名稱:")
    id := uuid.New()
    fmt.Println("UUID:", id.String()) // 輸出: UUID: [UUID字符串]
    
    // 使用簡化的 protobuf 別名
    fmt.Println("使用 protobuf (pb 別名)") // 輸出: 使用 protobuf (pb 別名)
    
    // 3. 版本區別
    fmt.Println("\n3. 版本區別:")
    v1Response := v1.GetUser(123)
    fmt.Println("V1 API 響應:", v1Response) // 輸出: V1 API 響應: v1 user data
    
    v2Response := v2.GetUser(123)
    fmt.Println("V2 API 響應:", v2Response) // 輸出: V2 API 響應: v2 user data
    
    // 4. 點匯入使用 (不推薦)
    fmt.Println("\n4. 點匯入使用:")
    result := ToUpper("hello world") // 直接使用 strings.ToUpper
    fmt.Println("轉大寫:", result) // 輸出: 轉大寫: HELLO WORLD
    
    // 5. 空白匯入效果
    fmt.Println("\n5. 空白匯入已註冊圖像解碼器") // 輸出: 空白匯入已註冊圖像解碼器
}

// 模擬 API v1
package v1

func GetUser(id int) string {
    return "v1 user data"
}

// 模擬 API v2  
package v2

func GetUser(id int) string {
    return "v2 user data"
}

### 程式包註解與 godoc

Go 提供了內建的文檔系統 godoc，通過特定格式的註解可以自動生成程式包文檔。良好的文檔是程式包可用性的重要組成部分。

**文檔註解規則**：
1. **程式包註解**：在 package 宣告前，以 "Package packagename" 開頭
2. **函式註解**：在函式宣告前，以函式名開頭
3. **類型註解**：在類型宣告前，以類型名開頭
4. **常數/變數註解**：在宣告前描述用途

**文檔撰寫最佳實踐**：
- 使用完整的句子
- 第一句話應該是簡潔的摘要
- 提供使用範例
- 說明參數和返回值
- 提及重要的注意事項

**godoc 工具**：
- `go doc package` - 查看程式包文檔
- `go doc package.Symbol` - 查看特定符號文檔
- `godoc -http=:6060` - 啟動本地文檔服務器

In [ ]:
/* godoc 文檔撰寫範例 */
package main

import "fmt"

// Package calculator 提供基本的數學運算功能。
//
// 這個程式包實現了常用的數學運算，包括加法、減法、乘法和除法。
// 所有函式都會進行適當的錯誤檢查，確保運算的安全性。
//
// 使用範例：
//     result := calculator.Add(10, 20)
//     fmt.Println(result) // 輸出: 30
package calculator

// Calculator 代表一個數學計算器。
//
// Calculator 結構體封裝了計算器的狀態，包括歷史記錄和當前結果。
// 它提供了鏈式調用的方式來進行連續運算。
type Calculator struct {
    result  float64   // 當前計算結果
    history []string  // 運算歷史記錄
}

// MaxValue 定義了計算器支持的最大數值。
//
// 超過此值的運算將返回錯誤，以防止溢出。
const MaxValue = 1e9

// DefaultPrecision 是默認的小數精度。
var DefaultPrecision = 2

// New 創建一個新的計算器實例。
//
// 返回的計算器已經初始化，可以立即使用。初始結果為 0。
//
// 範例：
//     calc := calculator.New()
//     result := calc.Add(10).Multiply(2).Result() // 結果: 20
func New() *Calculator {
    fmt.Println("創建新的計算器") // 輸出: 創建新的計算器
    return &Calculator{
        result:  0,
        history: make([]string, 0),
    }
}

// Add 將指定的數值加到當前結果。
//
// 參數 value 是要加的數值，可以是正數、負數或零。
// 返回計算器本身，支持鏈式調用。
func (c *Calculator) Add(value float64) *Calculator {
    c.result += value
    c.history = append(c.history, fmt.Sprintf("+ %.2f", value))
    fmt.Printf("執行加法: %.2f, 當前結果: %.2f\n", value, c.result)
    // 輸出: 執行加法: 10.00, 當前結果: 10.00
    return c
}

// Multiply 將當前結果乘以指定的數值。
//
// 參數 value 是乘數。如果 value 為 0，結果將變為 0。
// 返回計算器本身，支持鏈式調用。
func (c *Calculator) Multiply(value float64) *Calculator {
    c.result *= value
    c.history = append(c.history, fmt.Sprintf("× %.2f", value))
    fmt.Printf("執行乘法: %.2f, 當前結果: %.2f\n", value, c.result)
    // 輸出: 執行乘法: 2.00, 當前結果: 20.00
    return c
}

// Divide 將當前結果除以指定的數值。
//
// 參數 value 是除數。如果 value 為 0，函式會 panic。
// 建議在調用前檢查除數是否為零。
// 返回計算器本身，支持鏈式調用。
func (c *Calculator) Divide(value float64) *Calculator {
    if value == 0 {
        panic("除數不能為零")
    }
    c.result /= value
    c.history = append(c.history, fmt.Sprintf("÷ %.2f", value))
    fmt.Printf("執行除法: %.2f, 當前結果: %.2f\n", value, c.result)
    // 輸出: 執行除法: 4.00, 當前結果: 5.00
    return c
}

// Result 返回當前的計算結果。
//
// 這個方法不會修改計算器的狀態，可以多次調用。
func (c *Calculator) Result() float64 {
    fmt.Printf("獲取當前結果: %.2f\n", c.result)
    // 輸出: 獲取當前結果: 5.00
    return c.result
}

// History 返回計算器的運算歷史。
//
// 返回值是一個字符串切片，包含了所有執行過的運算。
// 歷史記錄按照執行順序排列。
func (c *Calculator) History() []string {
    fmt.Println("運算歷史:", c.history) // 輸出: 運算歷史: [+ 10.00 × 2.00 ÷ 4.00]
    return c.history
}

// Add 是一個便捷函式，直接執行兩個數的加法。
//
// 這是一個獨立函式，不需要創建計算器實例。
// 參數 a 和 b 是要相加的兩個數值。
// 返回兩數之和。
//
// 範例：
//     sum := calculator.Add(5, 3) // 返回 8
func Add(a, b float64) float64 {
    result := a + b
    fmt.Printf("計算 %.2f + %.2f = %.2f\n", a, b, result)
    // 輸出: 計算 5.00 + 3.00 = 8.00
    return result
}

func main() {
    fmt.Println("godoc 文檔範例")
    
    // 1. 使用靜態函式
    sum := Add(5, 3)
    fmt.Println("加法結果:", sum) // 輸出: 加法結果: 8
    
    // 2. 使用計算器實例
    calc := New()
    result := calc.Add(10).Multiply(2).Divide(4).Result()
    fmt.Println("鏈式計算結果:", result) // 輸出: 鏈式計算結果: 5
    
    // 3. 查看歷史記錄
    calc.History()
    
    fmt.Println("\n查看文檔命令:")
    fmt.Println("go doc calculator") // 輸出: go doc calculator
    fmt.Println("go doc calculator.Add") // 輸出: go doc calculator.Add
    fmt.Println("go doc calculator.Calculator") // 輸出: go doc calculator.Calculator
}

### internal 程式包

internal 程式包是 Go 提供的特殊機制，用於創建只能被特定範圍內的代碼匯入的程式包。這提供了比小寫標識符號更強的封裝性。

**internal 規則**：
1. **路徑限制**：包含 internal 目錄的程式包只能被其父路徑下的程式包匯入
2. **不可外部匯入**：外部模組無法匯入 internal 程式包
3. **範圍控制**：提供模組級別的訪問控制

**使用場景**：
- 內部實現細節
- 共享的內部工具
- 防止外部依賴內部實現
- 大型項目的模組化

**目錄結構範例**：
```
myproject/
├── cmd/
├── internal/          # 內部程式包根目錄
│   ├── auth/         # 內部認證邏輯
│   ├── database/     # 內部數據庫封裝
│   └── config/       # 內部配置管理
└── pkg/              # 公開程式包
```

In [ ]:
/* internal 程式包的應用 */
package main

import "fmt"

// 目錄結構示例：
/*
myproject/
├── go.mod
├── main.go              # ✓ 可以匯入 internal 程式包
├── cmd/
│   └── server/
│       └── main.go      # ✓ 可以匯入 internal 程式包
├── internal/            # 內部程式包根目錄
│   ├── auth/           
│   │   ├── jwt.go      # 內部 JWT 實現
│   │   └── session.go  # 內部會話管理
│   ├── database/       
│   │   ├── postgres.go # 內部數據庫實現
│   │   └── migrate.go  # 內部數據遷移
│   └── config/         
│       └── env.go      # 內部配置管理
├── pkg/                # 公開程式包
│   └── api/           
│       └── client.go   # ✓ 可以匯入 internal 程式包
└── external-project/   # 外部項目
    └── main.go         # ✗ 無法匯入 internal 程式包
*/

// 1. internal/auth/jwt.go - 內部認證邏輯
package auth

import (
    "fmt"
    "time"
)

type JWTManager struct {
    secret string
}

func NewJWTManager(secret string) *JWTManager {
    fmt.Println("創建內部 JWT 管理器") // 輸出: 創建內部 JWT 管理器
    return &JWTManager{secret: secret}
}

func (j *JWTManager) GenerateToken(userID int) string {
    fmt.Printf("為用戶 %d 生成內部 JWT 令牌\n", userID)
    // 輸出: 為用戶 123 生成內部 JWT 令牌
    return fmt.Sprintf("jwt_%d_%d", userID, time.Now().Unix())
}

func (j *JWTManager) ValidateToken(token string) bool {
    fmt.Println("驗證內部 JWT 令牌:", token)
    // 輸出: 驗證內部 JWT 令牌: jwt_123_1234567890
    return len(token) > 10 // 簡單驗證
}

// 2. internal/database/postgres.go - 內部數據庫邏輯
package database

import "fmt"

type PostgresDB struct {
    connectionString string
}

func NewPostgresDB(connStr string) *PostgresDB {
    fmt.Println("創建內部 PostgreSQL 連接") // 輸出: 創建內部 PostgreSQL 連接
    return &PostgresDB{connectionString: connStr}
}

func (db *PostgresDB) Connect() error {
    fmt.Println("連接到內部數據庫") // 輸出: 連接到內部數據庫
    return nil
}

func (db *PostgresDB) ExecuteQuery(sql string) []map[string]interface{} {
    fmt.Println("執行內部 SQL 查詢:", sql)
    // 輸出: 執行內部 SQL 查詢: SELECT * FROM users
    return []map[string]interface{}{
        {"id": 1, "name": "Alice"},
    }
}

// 3. internal/config/env.go - 內部配置管理
package config

import (
    "fmt"
    "os"
)

type Config struct {
    DatabaseURL string
    JWTSecret   string
    Port        string
}

func LoadConfig() *Config {
    fmt.Println("加載內部配置") // 輸出: 加載內部配置
    
    return &Config{
        DatabaseURL: getEnv("DATABASE_URL", "localhost:5432"),
        JWTSecret:   getEnv("JWT_SECRET", "default-secret"),
        Port:        getEnv("PORT", "8080"),
    }
}

func getEnv(key, defaultValue string) string {
    if value := os.Getenv(key); value != "" {
        return value
    }
    return defaultValue
}

// 4. main.go - 主應用程式（可以使用 internal 程式包）
package main

import (
    "fmt"
    // 可以匯入同一模組的 internal 程式包
    "myproject/internal/auth"
    "myproject/internal/database"
    "myproject/internal/config"
)

func main() {
    fmt.Println("internal 程式包使用示例")
    
    // 使用內部配置
    cfg := config.LoadConfig()
    fmt.Printf("配置端口: %s\n", cfg.Port) // 輸出: 配置端口: 8080
    
    // 使用內部數據庫
    db := database.NewPostgresDB(cfg.DatabaseURL)
    db.Connect()
    results := db.ExecuteQuery("SELECT * FROM users")
    fmt.Println("查詢結果:", results[0]["name"]) // 輸出: 查詢結果: Alice
    
    // 使用內部認證
    jwt := auth.NewJWTManager(cfg.JWTSecret)
    token := jwt.GenerateToken(123)
    fmt.Println("生成令牌:", token) // 輸出: 生成令牌: jwt_123_1234567890
    
    isValid := jwt.ValidateToken(token)
    fmt.Println("令牌有效:", isValid) // 輸出: 令牌有效: true
    
    fmt.Println("\ninternal 程式包的優點:")
    fmt.Println("- 防止外部依賴內部實現") // 輸出: - 防止外部依賴內部實現
    fmt.Println("- 提供模組級別的封裝") // 輸出: - 提供模組級別的封裝
    fmt.Println("- 便於重構內部結構") // 輸出: - 便於重構內部結構
}

### init 函式：請量避免

init 函式是 Go 中的特殊函式，在程式包被匯入時自動執行。雖然功能強大，但應該謹慎使用，因為它會增加代碼的複雜性和不可預測性。

**init 函式特性**：
1. **自動執行**：程式包匯入時自動調用，無需顯式呼叫
2. **執行順序**：按照依賴關係和匯入順序執行
3. **多個 init**：一個程式包可以有多個 init 函式
4. **無參數無返回值**：不能有參數或返回值

**適用場景（有限）**：
- 註冊驅動程式（如數據庫驅動）
- 初始化全局變數
- 一次性的設置工作

**為什麼要避免**：
- 隱式執行，難以控制
- 增加測試難度
- 影響程式啟動性能
- 難以調試和理解

**替代方案**：
- 使用顯式的初始化函式
- 延遲初始化
- 依賴注入

In [ ]:
/* init 函式的使用與注意事項 */
package main

import (
    "fmt"
    "log"
    "os"
)

// 1. init 函式的基本使用（不推薦的例子）

var globalConfig map[string]string

// 第一個 init 函式
func init() {
    fmt.Println("第一個 init 函式執行") // 輸出: 第一個 init 函式執行
    globalConfig = make(map[string]string)
    globalConfig["app_name"] = "MyApp"
}

// 第二個 init 函式（同一個程式包可以有多個）
func init() {
    fmt.Println("第二個 init 函式執行") // 輸出: 第二個 init 函式執行
    globalConfig["version"] = "1.0.0"
}

// 2. 常見的 init 使用場景（數據庫驅動註冊）
func init() {
    fmt.Println("註冊數據庫驅動") // 輸出: 註冊數據庫驅動
    // 模擬驅動註冊
    // sql.Register("postgres", &PostgresDriver{})
}

// 3. 有問題的 init 使用（會造成問題）

var logger *log.Logger

func init() {
    fmt.Println("危險的 init 函式") // 輸出: 危險的 init 函式
    
    // 問題：依賴外部資源，可能失敗
    file, err := os.OpenFile("app.log", os.O_CREATE|os.O_WRONLY|os.O_APPEND, 0666)
    if err != nil {
        // init 函式中的錯誤處理很困難
        log.Fatalf("無法創建日誌文件: %v", err)
    }
    logger = log.New(file, "APP: ", log.LstdFlags)
}

// 4. 更好的替代方案 - 顯式初始化

type Config struct {
    AppName string
    Version string
}

// 替代方案 1：顯式初始化函式
func NewConfig() *Config {
    fmt.Println("顯式創建配置") // 輸出: 顯式創建配置
    return &Config{
        AppName: "MyApp",
        Version: "1.0.0",
    }
}

// 替代方案 2：延遲初始化
type Logger struct {
    logger *log.Logger
    once   sync.Once
}

func (l *Logger) getLogger() *log.Logger {
    l.once.Do(func() {
        fmt.Println("延遲初始化日誌器") // 輸出: 延遲初始化日誌器
        file, err := os.OpenFile("app.log", os.O_CREATE|os.O_WRONLY|os.O_APPEND, 0666)
        if err != nil {
            l.logger = log.New(os.Stderr, "APP: ", log.LstdFlags)
            return
        }
        l.logger = log.New(file, "APP: ", log.LstdFlags)
    })
    return l.logger
}

func (l *Logger) Info(message string) {
    logger := l.getLogger()
    logger.Println("INFO:", message)
}

// 替代方案 3：依賴注入
type DatabaseManager struct {
    db Database
}

func NewDatabaseManager(db Database) *DatabaseManager {
    fmt.Println("通過依賴注入創建數據庫管理器") // 輸出: 通過依賴注入創建數據庫管理器
    return &DatabaseManager{db: db}
}

func main() {
    fmt.Println("init 函式示例開始")
    
    // init 函式已經執行完畢，全局變數已初始化
    fmt.Println("全局配置:", globalConfig) // 輸出: 全局配置: map[app_name:MyApp version:1.0.0]
    
    fmt.Println("\n使用更好的替代方案:")
    
    // 1. 顯式初始化
    config := NewConfig()
    fmt.Printf("配置: %+v\n", config) // 輸出: 配置: {AppName:MyApp Version:1.0.0}
    
    // 2. 延遲初始化
    logger := &Logger{}
    logger.Info("這是一條測試日誌")
    
    // 3. 依賴注入
    db := &MockDatabase{}
    dbManager := NewDatabaseManager(db)
    fmt.Printf("數據庫管理器創建完成: %T\n", dbManager)
    // 輸出: 數據庫管理器創建完成: *main.DatabaseManager
    
    fmt.Println("\ninit 函式的問題:")
    fmt.Println("- 隱式執行，難以控制時機") // 輸出: - 隱式執行，難以控制時機
    fmt.Println("- 增加測試難度") // 輸出: - 增加測試難度
    fmt.Println("- 錯誤處理困難") // 輸出: - 錯誤處理困難
    fmt.Println("- 影響程式啟動性能") // 輸出: - 影響程式啟動性能
    
    fmt.Println("\n推薦的替代方案:")
    fmt.Println("- 使用顯式初始化函式") // 輸出: - 使用顯式初始化函式
    fmt.Println("- 延遲初始化（sync.Once）") // 輸出: - 延遲初始化（sync.Once）
    fmt.Println("- 依賴注入") // 輸出: - 依賴注入
}

### 循環依賴關係

循環依賴是指兩個或多個程式包相互依賴的情況。Go 編譯器禁止循環依賴，這種設計強制開發者創建更清晰的架構。

**循環依賴的類型**：
1. **直接循環依賴**：A 依賴 B，B 依賴 A
2. **間接循環依賴**：A → B → C → A

**循環依賴的問題**：
- 編譯錯誤，無法構建
- 代碼耦合度過高
- 難以理解和維護
- 影響測試和重構

**解決策略**：
1. **提取公共介面**：創建共享的介面程式包
2. **依賴反轉**：讓高層模組定義介面，低層模組實現
3. **中介者模式**：引入中間層協調依賴關係
4. **重構程式包結構**：合併相關功能或拆分程式包

**預防措施**：
- 清晰的分層架構
- 單向依賴流
- 介面導向設計

In [ ]:
/* 循環依賴問題與解決方法 */
package main

import "fmt"

// 1. 循環依賴問題演示（會造成編譯錯誤）

/*
// 檔案: user/user.go
package user

import "myproject/order" // 引入 order 程式包

type User struct {
    ID   int
    Name string
}

func (u *User) GetOrders() []order.Order {
    // 用戶想要獲取自己的訂單
    return order.GetOrdersByUser(u.ID)
}

// 檔案: order/order.go  
package order

import "myproject/user" // 引入 user 程式包 - 造成循環依賴！

type Order struct {
    ID     int
    UserID int
}

func (o *Order) GetUser() *user.User {
    // 訂單想要獲取用戶信息
    return user.GetUserByID(o.UserID)
}

func GetOrdersByUser(userID int) []Order {
    // 實現略
    return nil
}

// 編譯錯誤: import cycle not allowed
*/

// 2. 解決方案 1：提取公共介面

// 檔案: types/types.go - 公共類型定義
package types

type User struct {
    ID   int    `json:"id"`
    Name string `json:"name"`
}

type Order struct {
    ID     int `json:"id"`
    UserID int `json:"user_id"`
}

// 檔案: interfaces/interfaces.go - 公共介面定義
package interfaces

import "myproject/types"

type UserService interface {
    GetUserByID(id int) (*types.User, error)
    GetUserOrders(userID int) ([]types.Order, error)
}

type OrderService interface {
    GetOrderByID(id int) (*types.Order, error)
    CreateOrder(userID int) (*types.Order, error)
}

// 3. 解決方案 2：依賴反轉

// 檔案: user/service.go - 用戶服務實現
package user

import (
    "fmt"
    "myproject/types"
    "myproject/interfaces"
)

type Service struct {
    orderService interfaces.OrderService // 依賴介面而不是具體實現
}

func NewService(orderService interfaces.OrderService) *Service {
    fmt.Println("創建用戶服務") // 輸出: 創建用戶服務
    return &Service{orderService: orderService}
}

func (s *Service) GetUserByID(id int) (*types.User, error) {
    fmt.Printf("獲取用戶 ID: %d\n", id) // 輸出: 獲取用戶 ID: 1
    return &types.User{ID: id, Name: "Alice"}, nil
}

func (s *Service) GetUserOrders(userID int) ([]types.Order, error) {
    fmt.Printf("獲取用戶 %d 的訂單\n", userID) // 輸出: 獲取用戶 1 的訂單
    
    // 通過介面調用訂單服務，沒有直接依賴
    order, err := s.orderService.CreateOrder(userID)
    if err != nil {
        return nil, err
    }
    
    return []types.Order{*order}, nil
}

// 檔案: order/service.go - 訂單服務實現  
package order

import (
    "fmt"
    "myproject/types"
    "myproject/interfaces"
)

type Service struct {
    userService interfaces.UserService // 依賴介面而不是具體實現
}

func NewService(userService interfaces.UserService) *Service {
    fmt.Println("創建訂單服務") // 輸出: 創建訂單服務
    return &Service{userService: userService}
}

func (s *Service) GetOrderByID(id int) (*types.Order, error) {
    fmt.Printf("獲取訂單 ID: %d\n", id) // 輸出: 獲取訂 ID: 123
    return &types.Order{ID: id, UserID: 1}, nil
}

func (s *Service) CreateOrder(userID int) (*types.Order, error) {
    fmt.Printf("為用戶 %d 創建訂單\n", userID) // 輸出: 為用戶 1 創建訂單
    
    // 通過介面驗證用戶存在，沒有直接依賴
    _, err := s.userService.GetUserByID(userID)
    if err != nil {
        return nil, err
    }
    
    order := &types.Order{ID: 123, UserID: userID}
    return order, nil
}

// 4. 解決方案 3：中介者模式

// 檔案: coordinator/coordinator.go - 協調器
package coordinator

import (
    "fmt"
    "myproject/types"
    "myproject/interfaces"
)

type BusinessCoordinator struct {
    userService  interfaces.UserService
    orderService interfaces.OrderService
}

func NewBusinessCoordinator(userSvc interfaces.UserService, orderSvc interfaces.OrderService) *BusinessCoordinator {
    fmt.Println("創建業務協調器") // 輸出: 創建業務協調器
    return &BusinessCoordinator{
        userService:  userSvc,
        orderService: orderSvc,
    }
}

func (bc *BusinessCoordinator) GetUserWithOrders(userID int) (*types.User, []types.Order, error) {
    fmt.Printf("協調獲取用戶 %d 及其訂單\n", userID) // 輸出: 協調獲取用戶 1 及其訂單
    
    // 協調兩個服務的調用
    user, err := bc.userService.GetUserByID(userID)
    if err != nil {
        return nil, nil, err
    }
    
    orders, err := bc.userService.GetUserOrders(userID)
    if err != nil {
        return nil, nil, err
    }
    
    return user, orders, nil
}

func main() {
    fmt.Println("循環依賴解決方案示例")
    
    // 使用依賴注入避免循環依賴
    fmt.Println("\n1. 依賴反轉方案:")
    
    // 創建服務實例（注意這裡需要小心處理依賴注入）
    var userSvc interfaces.UserService
    var orderSvc interfaces.OrderService
    
    // 模擬創建服務（實際中可能需要依賴注入容器）
    userSvc = user.NewService(nil) // 先創建不完整的實例
    orderSvc = order.NewService(userSvc)
    
    // 重新創建完整的用戶服務
    userSvc = user.NewService(orderSvc)
    
    fmt.Println("\n2. 中介者模式:")
    coordinator := coordinator.NewBusinessCoordinator(userSvc, orderSvc)
    
    user, orders, err := coordinator.GetUserWithOrders(1)
    if err != nil {
        fmt.Println("錯誤:", err)
        return
    }
    
    fmt.Printf("用戶: %+v\n", user) // 輸出: 用戶: &{ID:1 Name:Alice}
    fmt.Printf("訂單數量: %d\n", len(orders)) // 輸出: 訂單數量: 1
    
    fmt.Println("\n循環依賴的解決要點:")
    fmt.Println("- 提取公共介面和類型") // 輸出: - 提取公共介面和類型
    fmt.Println("- 使用依賴反轉原則") // 輸出: - 使用依賴反轉原則
    fmt.Println("- 引入中介者協調依賴") // 輸出: - 引入中介者協調依賴
    fmt.Println("- 保持單向依賴流") // 輸出: - 保持單向依賴流
}

### 優雅地重命名與新組織你的 API

隨著項目的發展，API 重構是不可避免的。Go 提供了一些技術和工具來幫助優雅地進行 API 重構，同時保持向後兼容性。

**API 重構的常見需求**：
1. **函式重命名**：提供更清晰的命名
2. **類型重構**：改進數據結構設計
3. **程式包重組**：更好的模組化結構
4. **版本管理**：支持多個 API 版本

**重構策略**：
1. **別名和包裝器**：為舊 API 提供別名或包裝器
2. **廢棄標記**：使用註解標記廢棄的 API
3. **分階段遷移**：逐步引入新 API，逐步移除舊 API
4. **版本分離**：並行維護多個版本

**工具支持**：
- `go fix` - 自動修復代碼
- `go doc` - 文檔生成
- 靜態分析工具 - 檢測 API 使用

In [ ]:
/* API 重構的最佳實踐 */
package main

import (
    "fmt"
    "time"
)

// 1. 函式重命名 - 使用別名保持兼容性

// 新的、更好的 API
func CalculateAge(birthYear int) int {
    currentYear := time.Now().Year()
    age := currentYear - birthYear
    fmt.Printf("計算年齡: %d - %d = %d\n", currentYear, birthYear, age)
    // 輸出: 計算年齡: 2024 - 1990 = 34
    return age
}

// 舊 API - 使用別名保持向後兼容
// Deprecated: Use CalculateAge instead.
func CalcAge(birthYear int) int {
    fmt.Println("使用已廢棄的 CalcAge 函式") // 輸出: 使用已廢棄的 CalcAge 函式
    return CalculateAge(birthYear) // 內部調用新函式
}

// 2. 類型重構 - 使用類型別名

// 新的用戶類型，有更好的結構
type User struct {
    ID        int       `json:"id"`
    FullName  string    `json:"full_name"`
    Email     string    `json:"email"`
    CreatedAt time.Time `json:"created_at"`
}

// 舊類型別名 - 保持兼容性
// Deprecated: Use User instead.
type UserInfo = User

// 新的創建函式
func NewUser(fullName, email string) *User {
    fmt.Printf("創建新用戶: %s (%s)\n", fullName, email)
    // 輸出: 創建新用戶: Alice Smith (alice@example.com)
    return &User{
        FullName:  fullName,
        Email:     email,
        CreatedAt: time.Now(),
    }
}

// 舊的創建函式 - 包裝新函式
// Deprecated: Use NewUser instead.
func CreateUserInfo(name, email string) *UserInfo {
    fmt.Println("使用已廢棄的 CreateUserInfo 函式") // 輸出: 使用已廢棄的 CreateUserInfo 函式
    return NewUser(name, email)
}

// 3. 程式包重組 - 重新導出

// 原本在 utils 程式包中的函式，現在移到專門的程式包

// 檔案: math/calculator.go - 新的專門程式包
package calculator

func Add(a, b int) int {
    result := a + b
    fmt.Printf("計算: %d + %d = %d\n", a, b, result)
    // 輸出: 計算: 10 + 20 = 30
    return result
}

func Multiply(a, b int) int {
    result := a * b
    fmt.Printf("計算: %d × %d = %d\n", a, b, result)
    // 輸出: 計算: 5 × 6 = 30
    return result
}

// 檔案: utils/utils.go - 舊程式包，重新導出新函式
package utils

import "myproject/calculator"

// Deprecated: Use calculator.Add instead.
func Add(a, b int) int {
    fmt.Println("使用 utils.Add（已廢棄）") // 輸出: 使用 utils.Add（已廢棄）
    return calculator.Add(a, b)
}

// Deprecated: Use calculator.Multiply instead.
func Multiply(a, b int) int {
    fmt.Println("使用 utils.Multiply（已廢棄）") // 輸出: 使用 utils.Multiply（已廢棄）
    return calculator.Multiply(a, b)
}

// 4. 版本管理 - 使用子程式包

// 檔案: api/v1/user.go - 版本 1 API
package v1

import "fmt"

type UserResponse struct {
    Name  string `json:"name"`
    Email string `json:"email"`
}

func GetUser(id int) *UserResponse {
    fmt.Printf("V1 API: 獲取用戶 %d\n", id) // 輸出: V1 API: 獲取用戶 123
    return &UserResponse{
        Name:  "Alice",
        Email: "alice@example.com",
    }
}

// 檔案: api/v2/user.go - 版本 2 API
package v2

import "fmt"

type UserResponse struct {
    ID       int    `json:"id"`
    FullName string `json:"full_name"`
    Email    string `json:"email"`
    Status   string `json:"status"`
}

func GetUser(id int) *UserResponse {
    fmt.Printf("V2 API: 獲取用戶 %d（增強版）\n", id) // 輸出: V2 API: 獲取用戶 123（增強版）
    return &UserResponse{
        ID:       id,
        FullName: "Alice Smith", 
        Email:    "alice@example.com",
        Status:   "active",
    }
}

// 5. 遷移助手 - 提供遷移工具

type MigrationHelper struct{}

func (m *MigrationHelper) ConvertV1ToV2(v1User *v1.UserResponse) *v2.UserResponse {
    fmt.Println("轉換 V1 用戶數據到 V2 格式") // 輸出: 轉換 V1 用戶數據到 V2 格式
    return &v2.UserResponse{
        ID:       1, // 需要從其他地方獲取
        FullName: v1User.Name,
        Email:    v1User.Email,
        Status:   "active", // 默認值
    }
}

func main() {
    fmt.Println("API 重構示例")
    
    // 1. 函式重命名示例
    fmt.Println("\n1. 函式重命名:")
    age1 := CalculateAge(1990) // 使用新 API
    age2 := CalcAge(1990)      // 使用舊 API（仍然有效）
    fmt.Printf("新 API 結果: %d, 舊 API 結果: %d\n", age1, age2)
    // 輸出: 新 API 結果: 34, 舊 API 結果: 34
    
    // 2. 類型重構示例
    fmt.Println("\n2. 類型重構:")
    user1 := NewUser("Alice Smith", "alice@example.com") // 使用新 API
    user2 := CreateUserInfo("Bob Jones", "bob@example.com") // 使用舊 API
    fmt.Printf("新 API 用戶: %s, 舊 API 用戶: %s\n", user1.FullName, user2.FullName)
    // 輸出: 新 API 用戶: Alice Smith, 舊 API 用戶: Bob Jones
    
    // 3. 程式包重組示例
    fmt.Println("\n3. 程式包重組:")
    result1 := calculator.Add(10, 20) // 使用新程式包
    result2 := utils.Add(10, 20)      // 使用舊程式包（仍然有效）
    fmt.Printf("新程式包結果: %d, 舊程式包結果: %d\n", result1, result2)
    // 輸出: 新程式包結果: 30, 舊程式包結果: 30
    
    // 4. 版本管理示例
    fmt.Println("\n4. 版本管理:")
    v1User := v1.GetUser(123)
    v2User := v2.GetUser(123)
    
    // 5. 遷移助手示例
    fmt.Println("\n5. 遷移助手:")
    helper := &MigrationHelper{}
    convertedUser := helper.ConvertV1ToV2(v1User)
    fmt.Printf("原始 V1: %+v\n", v1User)
    fmt.Printf("轉換後 V2: %+v\n", convertedUser)
    
    fmt.Println("\nAPI 重構最佳實踐:")
    fmt.Println("- 使用 Deprecated 註解標記廢棄 API") // 輸出: - 使用 Deprecated 註解標記廢棄 API
    fmt.Println("- 提供別名和包裝器保持兼容性") // 輸出: - 提供別名和包裝器保持兼容性
    fmt.Println("- 使用版本化程式包管理不兼容變更") // 輸出: - 使用版本化程式包管理不兼容變更
    fmt.Println("- 提供遷移工具和文檔") // 輸出: - 提供遷移工具和文檔
}

## 使用模組

Go 模組系統是現代 Go 開發的核心，提供了依賴管理、版本控制和程式包分發的完整解決方案。理解如何正確使用模組對於構建可維護的 Go 應用程式至關重要。

**模組系統的優勢**：
1. **依賴管理**：自動解析和管理第三方依賴
2. **版本控制**：語義化版本管理，確保相容性
3. **構建重現性**：固定依賴版本，確保一致的構建結果
4. **安全性**：檢驗模組完整性和真實性

**模組的核心概念**：
- **模組路徑**：唯一標識模組的路徑，通常是版本庫 URL
- **版本**：模組的特定版本，遵循語義化版本規則
- **依賴圖**：模組間的依賴關係樹
- **最小版本選擇**：選擇滿足所有約束的最低版本

In [ ]:
/* 模組使用基本概念 */
package main

import (
    "fmt"
    "os"
    "os/exec"
)

func main() {
    fmt.Println("Go 模組系統基本概念")
    
    // 1. 模組初始化
    fmt.Println("\n1. 初始化新模組:")
    fmt.Println("go mod init github.com/example/myproject")
    fmt.Println("創建 go.mod 檔案，定義模組身份") // 輸出: 創建 go.mod 檔案，定義模組身份
    
    // 2. go.mod 檔案結構示例
    fmt.Println("\n2. go.mod 檔案內容:")
    goModContent := `module github.com/example/myproject

go 1.19

require (
    github.com/gin-gonic/gin v1.9.1
    github.com/gorilla/mux v1.8.0
    gorm.io/gorm v1.25.1
)

require (
    github.com/bytedance/sonic v1.8.8 // indirect
    github.com/chenzhuoyu/base64x v0.0.0-20221115062448-fe3a3abad311 // indirect
    github.com/gin-contrib/sse v0.1.0 // indirect
    // ... 更多間接依賴
)

replace (
    github.com/old/package => github.com/new/package v1.0.0
)

exclude (
    github.com/broken/package v1.2.3
)`
    
    fmt.Println(goModContent)
    
    // 3. 模組操作命令
    fmt.Println("\n3. 常用模組命令:")
    
    commands := []struct {
        command string
        description string
    }{
        {"go mod init <module-name>", "初始化新模組"},
        {"go mod tidy", "添加缺少的依賴，移除未使用的依賴"},
        {"go mod download", "下載依賴到本地快取"},
        {"go mod verify", "驗證依賴的完整性"},
        {"go mod graph", "打印模組依賴圖"},
        {"go mod why <package>", "解釋為什麼需要某個依賴"},
        {"go mod vendor", "複製依賴到 vendor 目錄"},
        {"go mod edit", "編輯 go.mod 檔案"},
    }
    
    for _, cmd := range commands {
        fmt.Printf("%-25s - %s\n", cmd.command, cmd.description)
        // 輸出: go mod init <module-name> - 初始化新模組
        // 輸出: go mod tidy            - 添加缺少的依賴，移除未使用的依賴
        // ... 等等
    }
    
    // 4. 模組版本格式
    fmt.Println("\n4. 模組版本格式:")
    versionFormats := []struct {
        format string
        example string
        description string
    }{
        {"vX.Y.Z", "v1.2.3", "語義化版本標籤"},
        {"vX.Y.Z-pre", "v1.2.3-alpha.1", "預發布版本"},
        {"vX.Y.Z-timestamp-commit", "v0.0.0-20230101120000-abcdef123456", "偽版本"},
        {"commit", "abcdef123456", "特定 commit"},
        {"branch", "main", "分支名稱"},
    }
    
    for _, ver := range versionFormats {
        fmt.Printf("%-25s %-35s %s\n", ver.format, ver.example, ver.description)
        // 輸出: vX.Y.Z                 v1.2.3                            語義化版本標籤
        // 輸出: vX.Y.Z-pre             v1.2.3-alpha.1                    預發布版本
        // ... 等等
    }
    
    // 5. 模組代理和檢驗
    fmt.Println("\n5. 模組代理和檢驗:")
    fmt.Println("GOPROXY=" + os.Getenv("GOPROXY"))
    fmt.Println("GOSUMDB=" + os.Getenv("GOSUMDB"))
    fmt.Println("GOPRIVATE=" + os.Getenv("GOPRIVATE"))
    
    // 模組代理說明
    fmt.Println("\n模組代理的作用:")
    fmt.Println("- 提供模組下載服務") // 輸出: - 提供模組下載服務
    fmt.Println("- 快取和加速下載") // 輸出: - 快取和加速下載
    fmt.Println("- 提供模組發現功能") // 輸出: - 提供模組發現功能
    fmt.Println("- 支持離線構建") // 輸出: - 支持離線構建
    
    // 6. go.sum 檔案
    fmt.Println("\n6. go.sum 檔案的作用:")
    fmt.Println("- 記錄模組的加密雜湊值") // 輸出: - 記錄模組的加密雜湊值
    fmt.Println("- 確保模組內容未被篡改") // 輸出: - 確保模組內容未被篡改
    fmt.Println("- 提供構建重現性") // 輸出: - 提供構建重現性
    fmt.Println("- 應該提交到版本控制") // 輸出: - 應該提交到版本控制
    
    goSumExample := `github.com/gin-gonic/gin v1.9.1 h1:4idEAncQnU5cB7BeOkPtxjfCSye0AAm1R0RVIqJ+Jmg=
github.com/gin-gonic/gin v1.9.1/go.mod h1:hPrL7YrpYKXt5YId3A/Tnip5kqbEAP+KLuI3SUcPTeU=`
    
    fmt.Println("\ngo.sum 檔案範例:")
    fmt.Println(goSumExample)
}

### 匯入第三方程式

第三方程式包是 Go 生態系統的重要組成部分，為開發者提供了豐富的功能和工具。正確使用第三方程式包可以大大提高開發效率。

**選擇第三方程式包的標準**：
1. **活躍維護**：檢查最後更新時間和 issue 響應速度
2. **文檔完善**：良好的文檔和範例
3. **測試覆蓋率**：充分的測試保證品質
4. **社群支持**：活躍的社群和貢獻者
5. **授權相容**：確認授權條款符合專案需求

**常見的第三方程式包類型**：
- **Web 框架**：Gin, Echo, Fiber
- **數據庫**：GORM, sqlx, MongoDB driver
- **HTTP 客戶端**：Resty, grab
- **日誌**：Logrus, Zap, Zerolog
- **配置管理**：Viper, envconfig
- **測試工具**：Testify, Ginkgo

**最佳實踐**：
- 定期更新依賴
- 監控安全漏洞
- 避免依賴地獄
- 優先選擇標準庫

In [ ]:
/* 第三方套件的引入與使用 */
package main

import (
    "fmt"
    "log"
    "net/http"
    
    // 1. Web 框架 - Gin
    "github.com/gin-gonic/gin"
    
    // 2. HTTP 客戶端 - Resty
    "github.com/go-resty/resty/v2"
    
    // 3. 日誌庫 - Logrus
    "github.com/sirupsen/logrus"
    
    // 4. 配置管理 - Viper
    "github.com/spf13/viper"
    
    // 5. UUID 生成
    "github.com/google/uuid"
    
    // 6. 時間處理 - now
    "github.com/jinzhu/now"
)

func main() {
    fmt.Println("第三方套件使用示例")
    
    // 1. 使用 Gin Web 框架
    fmt.Println("\n1. Gin Web 框架:")
    setupGinServer()
    
    // 2. 使用 Resty HTTP 客戶端
    fmt.Println("\n2. Resty HTTP 客戶端:")
    useRestyClient()
    
    // 3. 使用 Logrus 日誌庫
    fmt.Println("\n3. Logrus 日誌庫:")
    useLogrus()
    
    // 4. 使用 Viper 配置管理
    fmt.Println("\n4. Viper 配置管理:")
    useViper()
    
    // 5. 使用 UUID 生成器
    fmt.Println("\n5. UUID 生成器:")
    useUUID()
    
    // 6. 使用 now 時間處理庫
    fmt.Println("\n6. now 時間處理:")
    useNowLibrary()
    
    // 7. 添加新依賴的流程
    fmt.Println("\n7. 添加第三方依賴的完整流程:")
    showDependencyWorkflow()
}

// 1. Gin Web 框架使用示例
func setupGinServer() {
    // 創建 Gin 路由器
    r := gin.Default()
    fmt.Println("創建 Gin 路由器") // 輸出: 創建 Gin 路由器
    
    // 添加路由
    r.GET("/ping", func(c *gin.Context) {
        fmt.Println("處理 /ping 請求") // 輸出: 處理 /ping 請求
        c.JSON(200, gin.H{
            "message": "pong",
        })
    })
    
    r.GET("/user/:id", func(c *gin.Context) {
        id := c.Param("id")
        fmt.Printf("獲取用戶 ID: %s\n", id) // 輸出: 獲取用戶 ID: 123
        c.JSON(200, gin.H{
            "id":   id,
            "name": "Alice",
        })
    })
    
    fmt.Println("Gin 服務器配置完成（未實際啟動）") // 輸出: Gin 服務器配置完成（未實際啟動）
    // 實際部署時使用: r.Run(":8080")
}

// 2. Resty HTTP 客戶端使用示例
func useRestyClient() {
    // 創建 Resty 客戶端
    client := resty.New()
    fmt.Println("創建 Resty HTTP 客戶端") // 輸出: 創建 Resty HTTP 客戶端
    
    // 設定基本配置
    client.SetBaseURL("https://api.example.com")
    client.SetHeader("User-Agent", "MyApp/1.0")
    client.SetTimeout(30 * time.Second)
    
    fmt.Println("配置完成:")
    fmt.Println("- 基礎 URL: https://api.example.com") // 輸出: - 基礎 URL: https://api.example.com
    fmt.Println("- 用戶代理: MyApp/1.0") // 輸出: - 用戶代理: MyApp/1.0
    fmt.Println("- 超時時間: 30 秒") // 輸出: - 超時時間: 30 秒
    
    // 模擬 HTTP 請求（實際會發送請求）
    /*
    resp, err := client.R().
        SetQueryParam("page", "1").
        SetHeader("Authorization", "Bearer token").
        Get("/users")
    
    if err != nil {
        log.Printf("請求失敗: %v", err)
        return
    }
    
    fmt.Printf("響應狀態: %d\n", resp.StatusCode())
    fmt.Printf("響應體: %s\n", resp.String())
    */
    
    fmt.Println("HTTP 客戶端就緒（未發送實際請求）") // 輸出: HTTP 客戶端就緒（未發送實際請求）
}

// 3. Logrus 日誌庫使用示例
func useLogrus() {
    // 配置 Logrus
    logrus.SetFormatter(&logrus.JSONFormatter{})
    logrus.SetLevel(logrus.InfoLevel)
    
    fmt.Println("配置 Logrus 日誌庫") // 輸出: 配置 Logrus 日誌庫
    
    // 使用結構化日誌
    logger := logrus.WithFields(logrus.Fields{
        "user_id": 123,
        "action":  "login",
    })
    
    logger.Info("用戶登錄成功")
    logger.Warn("用戶嘗試訪問受限資源")
    logger.Error("數據庫連接失敗")
    
    // 創建子日誌器
    apiLogger := logrus.WithField("component", "api")
    apiLogger.Info("API 服務器啟動")
    
    fmt.Println("日誌記錄完成") // 輸出: 日誌記錄完成
}

// 4. Viper 配置管理使用示例
func useViper() {
    fmt.Println("使用 Viper 配置管理") // 輸出: 使用 Viper 配置管理
    
    // 設定配置文件
    viper.SetConfigName("config")
    viper.SetConfigType("yaml")
    viper.AddConfigPath(".")
    viper.AddConfigPath("/etc/myapp/")
    
    // 設定環境變數
    viper.AutomaticEnv()
    viper.SetEnvPrefix("MYAPP")
    
    // 設定默認值
    viper.SetDefault("port", 8080)
    viper.SetDefault("debug", false)
    viper.SetDefault("database.host", "localhost")
    
    // 模擬讀取配置（實際會讀取檔案）
    /*
    if err := viper.ReadInConfig(); err != nil {
        log.Printf("配置檔案讀取失敗: %v", err)
    }
    */
    
    // 讀取配置值
    port := viper.GetInt("port")
    debug := viper.GetBool("debug")
    dbHost := viper.GetString("database.host")
    
    fmt.Printf("配置讀取完成:\n")
    fmt.Printf("- 端口: %d\n", port) // 輸出: - 端口: 8080
    fmt.Printf("- 調試模式: %t\n", debug) // 輸出: - 調試模式: false
    fmt.Printf("- 數據庫主機: %s\n", dbHost) // 輸出: - 數據庫主機: localhost
}

// 5. UUID 生成器使用示例
func useUUID() {
    fmt.Println("使用 Google UUID 庫") // 輸出: 使用 Google UUID 庫
    
    // 生成不同類型的 UUID
    uuid1 := uuid.New()
    uuid4 := uuid.Must(uuid.NewRandom())
    
    fmt.Printf("UUID v1: %s\n", uuid1.String()) // 輸出: UUID v1: [UUID字符串]
    fmt.Printf("UUID v4: %s\n", uuid4.String()) // 輸出: UUID v4: [UUID字符串]
    
    // 解析 UUID 字符串
    parsed, err := uuid.Parse(uuid1.String())
    if err != nil {
        log.Printf("UUID 解析失敗: %v", err)
        return
    }
    
    fmt.Printf("解析成功: %s\n", parsed.String()) // 輸出: 解析成功: [相同的UUID字符串]
    fmt.Printf("UUID 版本: %d\n", parsed.Version()) // 輸出: UUID 版本: 1 或 4
}

// 6. now 時間處理庫使用示例
func useNowLibrary() {
    fmt.Println("使用 now 時間處理庫") // 輸出: 使用 now 時間處理庫
    
    // 獲取各種時間點
    beginningOfWeek := now.BeginningOfWeek()
    endOfWeek := now.EndOfWeek()
    beginningOfMonth := now.BeginningOfMonth()
    endOfMonth := now.EndOfMonth()
    
    fmt.Printf("本週開始: %s\n", beginningOfWeek.Format("2006-01-02 15:04:05"))
    fmt.Printf("本週結束: %s\n", endOfWeek.Format("2006-01-02 15:04:05"))
    fmt.Printf("本月開始: %s\n", beginningOfMonth.Format("2006-01-02 15:04:05"))
    fmt.Printf("本月結束: %s\n", endOfMonth.Format("2006-01-02 15:04:05"))
    
    // 解析時間字符串
    parsedTime := now.MustParse("2023-12-25")
    fmt.Printf("聖誕節: %s\n", parsedTime.Format("2006-01-02"))
    
    fmt.Println("時間處理完成") // 輸出: 時間處理完成
}

// 7. 第三方依賴添加流程
func showDependencyWorkflow() {
    workflow := []string{
        "1. 在代碼中引入程式包",
        "2. 運行 'go mod tidy' 自動添加依賴",
        "3. 檢查 go.mod 和 go.sum 檔案變更",
        "4. 測試新功能是否正常工作",
        "5. 提交 go.mod 和 go.sum 到版本控制",
    }
    
    for _, step := range workflow {
        fmt.Println(step) // 輸出每個步驟
    }
    
    fmt.Println("\n手動添加依賴的命令:")
    fmt.Println("go get github.com/gin-gonic/gin@v1.9.1") // 輸出: go get github.com/gin-gonic/gin@v1.9.1
    fmt.Println("go get github.com/sirupsen/logrus@latest") // 輸出: go get github.com/sirupsen/logrus@latest
    
    fmt.Println("\n依賴管理最佳實踐:")
    fmt.Println("- 選擇維護活躍的程式包") // 輸出: - 選擇維護活躍的程式包
    fmt.Println("- 定期更新依賴版本") // 輸出: - 定期更新依賴版本
    fmt.Println("- 監控安全漏洞") // 輸出: - 監控安全漏洞
    fmt.Println("- 避免不必要的依賴") // 輸出: - 避免不必要的依賴
}

### 使用版本

Go 模組系統使用語義化版本（Semantic Versioning）來管理依賴版本，這提供了清晰的版本演進規則和相容性保證。

**語義化版本格式**：`vMAJOR.MINOR.PATCH[-PRERELEASE][+BUILD]`
- **MAJOR**：不相容的 API 變更
- **MINOR**：向後相容的功能新增
- **PATCH**：向後相容的錯誤修復
- **PRERELEASE**：預發布版本標識
- **BUILD**：構建元數據

**版本比較規則**：
1. 主版本號優先級最高
2. 次版本號次之
3. 補丁版本號最低
4. 預發布版本低於正式版本

**Go 模組的特殊規則**：
- v0.x.x 被認為是不穩定版本
- v1.x.x 開始遵循相容性承諾
- v2+ 需要在模組路徑中包含版本號

**版本選擇策略**：
- 使用具體版本：`v1.2.3`
- 使用版本範圍：`v1.2` 或 `v1`
- 使用最新版本：`latest`
- 使用分支或 commit：`main` 或 `abc123`

In [ ]:
/* 版本管理的實務 */
package main

import (
    "fmt"
    "sort"
    "strings"
)

// 版本結構體
type Version struct {
    Major      int
    Minor      int
    Patch      int
    Prerelease string
    Original   string
}

func main() {
    fmt.Println("Go 模組版本管理實務")
    
    // 1. 語義化版本示例
    fmt.Println("\n1. 語義化版本示例:")
    demonstrateSemanticVersioning()
    
    // 2. 版本選擇命令
    fmt.Println("\n2. 版本選擇命令:")
    showVersionCommands()
    
    // 3. 版本約束範例
    fmt.Println("\n3. 版本約束範例:")
    showVersionConstraints()
    
    // 4. 預發布版本處理
    fmt.Println("\n4. 預發布版本處理:")
    handlePrereleaseVersions()
    
    // 5. 版本升級策略
    fmt.Println("\n5. 版本升級策略:")
    showUpgradeStrategies()
}

func demonstrateSemanticVersioning() {
    versions := []string{
        "v1.0.0",         // 初始穩定版本
        "v1.0.1",         // 錯誤修復
        "v1.1.0",         // 新功能（向後相容）
        "v1.1.1",         // 錯誤修復
        "v2.0.0",         // 重大變更（不相容）
        "v2.1.0",         // 新功能
        "v2.1.1",         // 錯誤修復
        "v3.0.0-alpha.1", // 預發布版本
        "v3.0.0-beta.1",  // Beta 版本
        "v3.0.0",         // 正式版本
    }
    
    for _, version := range versions {
        description := getVersionDescription(version)
        fmt.Printf("%-15s - %s\n", version, description)
    }
}

func getVersionDescription(version string) string {
    switch {
    case strings.Contains(version, "-alpha"):
        return "Alpha 預發布版本，可能不穩定"
    case strings.Contains(version, "-beta"):
        return "Beta 預發布版本，功能基本完整"
    case strings.HasSuffix(version, ".0.0"):
        return "主版本更新，可能包含不相容變更"
    case strings.Contains(version, ".0"):
        if !strings.HasSuffix(version, ".0.0") {
            return "次版本更新，新增功能但向後相容"
        }
    default:
        return "補丁更新，錯誤修復"
    }
    return "標準版本"
}

func showVersionCommands() {
    commands := []struct {
        command     string
        description string
        example     string
    }{
        {
            "go get package@version",
            "獲取指定版本",
            "go get github.com/gin-gonic/gin@v1.9.1",
        },
        {
            "go get package@latest",
            "獲取最新穩定版本",
            "go get github.com/gin-gonic/gin@latest",
        },
        {
            "go get package@upgrade",
            "升級到相容的最新版本",
            "go get github.com/gin-gonic/gin@upgrade",
        },
        {
            "go get package@patch",
            "升級到最新補丁版本",
            "go get github.com/gin-gonic/gin@patch",
        },
        {
            "go get package@commit",
            "獲取特定 commit",
            "go get github.com/gin-gonic/gin@abc123",
        },
        {
            "go list -m -versions package",
            "列出可用版本",
            "go list -m -versions github.com/gin-gonic/gin",
        },
    }
    
    for _, cmd := range commands {
        fmt.Printf("命令: %s\n", cmd.command)
        fmt.Printf("說明: %s\n", cmd.description)
        fmt.Printf("範例: %s\n\n", cmd.example)
    }
}

func showVersionConstraints() {
    constraints := []struct {
        constraint  string
        meaning     string
        example     string
    }{
        {
            "v1.2.3",
            "精確版本",
            "只使用 v1.2.3",
        },
        {
            "v1.2",
            "次版本約束",
            "使用 v1.2.x 系列最新版本",
        },
        {
            "v1",
            "主版本約束",
            "使用 v1.x.x 系列最新版本",
        },
        {
            ">= v1.2.0",
            "最低版本約束",
            "至少 v1.2.0 或更新",
        },
        {
            "< v2.0.0",
            "最高版本約束",
            "小於 v2.0.0",
        },
    }
    
    for _, constraint := range constraints {
        fmt.Printf("約束: %-12s 含義: %-20s 說明: %s\n",
            constraint.constraint,
            constraint.meaning,
            constraint.example)
    }
    
    fmt.Println("\n版本選擇實例:")
    fmt.Println("如果依賴圖要求:")
    fmt.Println("- 模組 A 需要 package v1.2.0") // 輸出: - 模組 A 需要 package v1.2.0
    fmt.Println("- 模組 B 需要 package v1.3.1") // 輸出: - 模組 B 需要 package v1.3.1
    fmt.Println("- 模組 C 需要 package v1.2.5") // 輸出: - 模組 C 需要 package v1.2.5
    fmt.Println("Go 會選擇: v1.3.1 (滿足所有約束的最高版本)") // 輸出: Go 會選擇: v1.3.1 (滿足所有約束的最高版本)
}

func handlePrereleaseVersions() {
    fmt.Println("預發布版本處理規則:")
    fmt.Println("1. 預發布版本不會被自動選擇") // 輸出: 1. 預發布版本不會被自動選擇
    fmt.Println("2. 需要明確指定預發布版本") // 輸出: 2. 需要明確指定預發布版本
    fmt.Println("3. 預發布版本按字典序比較") // 輸出: 3. 預發布版本按字典序比較
    
    prereleaseVersions := []string{
        "v2.0.0-alpha.1",
        "v2.0.0-alpha.2",
        "v2.0.0-beta.1",
        "v2.0.0-rc.1",
        "v2.0.0",
    }
    
    fmt.Println("\n預發布版本順序:")
    for i, version := range prereleaseVersions {
        status := ""
        if i == len(prereleaseVersions)-1 {
            status = " (正式版本)"
        }
        fmt.Printf("%s%s\n", version, status)
    }
    
    fmt.Println("\n使用預發布版本:")
    fmt.Println("go get package@v2.0.0-beta.1") // 輸出: go get package@v2.0.0-beta.1
}

func showUpgradeStrategies() {
    strategies := []struct {
        strategy    string
        command     string
        description string
        risk        string
    }{
        {
            "保守升級",
            "go get package@patch",
            "只更新補丁版本",
            "風險最低，只修復錯誤",
        },
        {
            "功能升級",
            "go get package@upgrade",
            "升級次版本，獲取新功能",
            "中等風險，可能有新 API",
        },
        {
            "最新升級",
            "go get package@latest",
            "升級到最新穩定版本",
            "較高風險，可能跨主版本",
        },
        {
            "主版本升級",
            "手動更新 import 路徑",
            "升級主版本",
            "最高風險，可能有重大變更",
        },
    }
    
    for _, strategy := range strategies {
        fmt.Printf("策略: %s\n", strategy.strategy)
        fmt.Printf("命令: %s\n", strategy.command)
        fmt.Printf("說明: %s\n", strategy.description)
        fmt.Printf("風險: %s\n\n", strategy.risk)
    }
    
    fmt.Println("升級最佳實踐:")
    fmt.Println("1. 查看變更日誌 (CHANGELOG)")  // 輸出: 1. 查看變更日誌 (CHANGELOG)
    fmt.Println("2. 在測試環境先驗證")        // 輸出: 2. 在測試環境先驗證  
    fmt.Println("3. 運行完整的測試套件")      // 輸出: 3. 運行完整的測試套件
    fmt.Println("4. 逐步升級，避免大跳躍")    // 輸出: 4. 逐步升級，避免大跳躍
    fmt.Println("5. 保持依賴版本記錄")        // 輸出: 5. 保持依賴版本記錄
}

### 選擇最小版本

Go 使用最小版本選擇（Minimum Version Selection, MVS）算法來解決依賴版本衝突。這個算法的核心理念是選擇滿足所有約束條件的最低版本，確保構建的可預測性和穩定性。

**MVS 的工作原理**：
1. **構建依賴圖**：收集所有直接和間接依賴
2. **收集版本約束**：每個模組對依賴的版本要求
3. **選擇最小版本**：對每個依賴，選擇滿足所有約束的最低版本
4. **驗證相容性**：確保選中的版本組合可以協同工作

**MVS 的優勢**：
- **可預測性**：相同的依賴圖總是產生相同的版本選擇
- **穩定性**：避免意外升級到未測試的版本
- **簡單性**：算法簡單，易於理解和調試
- **向前相容**：新增依賴不會影響現有依賴的版本選擇

**與其他算法的比較**：
- **NPM/Yarn**：選擇最高相容版本，可能不穩定
- **Maven**：最近宣告獲勝，可能有意外
- **Go MVS**：選擇最小版本，最穩定可預測

In [ ]:
/* 最小版本選擇原理 */
package main

import (
    "fmt"
    "sort"
    "strings"
)

// 依賴關係結構
type Dependency struct {
    Name    string
    Version string
    Deps    []Dependency
}

// 版本約束
type VersionConstraint struct {
    Package string
    Version string
    Source  string
}

func main() {
    fmt.Println("最小版本選擇（MVS）算法示例")
    
    // 1. MVS 算法演示
    fmt.Println("\n1. MVS 算法工作原理:")
    demonstrateMVS()
    
    // 2. 版本衝突解決
    fmt.Println("\n2. 版本衝突解決:")
    resolveVersionConflicts()
    
    // 3. 與其他算法比較
    fmt.Println("\n3. 與其他依賴解析算法比較:")
    compareAlgorithms()
    
    // 4. MVS 實際應用
    fmt.Println("\n4. MVS 實際應用:")
    showMVSInPractice()
}

func demonstrateMVS() {
    fmt.Println("假設的依賴關係圖:")
    
    // 主模組的依賴
    fmt.Println("主模組 (main):")
    fmt.Println("├── A v1.2.0") // 輸出: ├── A v1.2.0
    fmt.Println("├── B v1.1.0") // 輸出: ├── B v1.1.0
    fmt.Println("└── C v1.0.0") // 輸出: └── C v1.0.0
    
    fmt.Println("\nA v1.2.0 的依賴:")
    fmt.Println("├── D v1.1.0") // 輸出: ├── D v1.1.0
    fmt.Println("└── E v1.2.0") // 輸出: └── E v1.2.0
    
    fmt.Println("\nB v1.1.0 的依賴:")
    fmt.Println("├── D v1.2.0") // 輸出: ├── D v1.2.0 (版本衝突!)
    fmt.Println("└── F v1.0.0") // 輸出: └── F v1.0.0
    
    fmt.Println("\nC v1.0.0 的依賴:")
    fmt.Println("└── E v1.1.0") // 輸出: └── E v1.1.0 (版本衝突!)
    
    fmt.Println("\n版本約束收集:")
    constraints := []VersionConstraint{
        {"D", "v1.1.0", "A v1.2.0"},
        {"D", "v1.2.0", "B v1.1.0"},
        {"E", "v1.2.0", "A v1.2.0"},
        {"E", "v1.1.0", "C v1.0.0"},
    }
    
    for _, constraint := range constraints {
        fmt.Printf("- %s 需要 %s %s\n", constraint.Source, constraint.Package, constraint.Version)
    }
    
    fmt.Println("\nMVS 選擇結果:")
    fmt.Println("- D: 選擇 v1.2.0 (滿足 v1.1.0 和 v1.2.0 約束的最小版本)") 
    // 輸出: - D: 選擇 v1.2.0 (滿足 v1.1.0 和 v1.2.0 約束的最小版本)
    fmt.Println("- E: 選擇 v1.2.0 (滿足 v1.1.0 和 v1.2.0 約束的最小版本)")
    // 輸出: - E: 選擇 v1.2.0 (滿足 v1.1.0 和 v1.2.0 約束的最小版本)
}

func resolveVersionConflicts() {
    scenarios := []struct {
        scenario     string
        requirements []string
        selected     string
        explanation  string
    }{
        {
            "簡單衝突",
            []string{"v1.1.0", "v1.2.0"},
            "v1.2.0",
            "v1.2.0 滿足兩個約束",
        },
        {
            "多重約束",
            []string{"v1.1.0", "v1.2.3", "v1.1.5"},
            "v1.2.3",
            "v1.2.3 是滿足所有約束的最小版本",
        },
        {
            "預發布版本",
            []string{"v1.1.0", "v2.0.0-beta.1"},
            "v2.0.0-beta.1",
            "預發布版本明確要求時會被選擇",
        },
        {
            "不相容版本",
            []string{"v1.2.0", "v2.1.0"},
            "錯誤",
            "主版本不相容，無法自動解決",
        },
    }
    
    for _, scenario := range scenarios {
        fmt.Printf("場景: %s\n", scenario.scenario)
        fmt.Printf("要求: %s\n", strings.Join(scenario.requirements, ", "))
        fmt.Printf("選擇: %s\n", scenario.selected)
        fmt.Printf("說明: %s\n\n", scenario.explanation)
    }
}

func compareAlgorithms() {
    fmt.Printf("%-15s %-20s %-30s %-20s\n", "算法", "策略", "優點", "缺點")
    fmt.Println(strings.Repeat("-", 85))
    
    algorithms := []struct {
        name     string
        strategy string
        pros     string
        cons     string
    }{
        {
            "Go MVS",
            "選擇最小滿足版本",
            "可預測、穩定、簡單",
            "可能錯過新功能",
        },
        {
            "NPM/Yarn",
            "選擇最高相容版本",
            "獲得最新功能和修復",
            "可能引入未測試版本",
        },
        {
            "Maven",
            "最近宣告獲勝",
            "解決衝突直觀",
            "順序依賴，不穩定",
        },
        {
            "Cargo",
            "語義化版本約束",
            "靈活的版本約束",
            "複雜度較高",
        },
    }
    
    for _, alg := range algorithms {
        fmt.Printf("%-15s %-20s %-30s %-20s\n",
            alg.name, alg.strategy, alg.pros, alg.cons)
    }
}

func showMVSInPractice() {
    fmt.Println("實際 MVS 應用場景:")
    
    scenarios := []struct {
        title       string
        situation   string
        mvsResult   string
        benefit     string
    }{
        {
            "新增依賴",
            "項目添加新的第三方庫",
            "不影響現有依賴版本",
            "保持構建穩定性",
        },
        {
            "間接依賴衝突",
            "兩個依賴要求同一庫的不同版本",
            "選擇滿足兩者的最小版本",
            "避免運行時衝突",
        },
        {
            "版本回退需求",
            "新版本有 bug 需要回退",
            "安全回退到較舊版本",
            "不會連鎖影響其他依賴",
        },
        {
            "大型項目管理",
            "多個子模組有複雜依賴",
            "一致的版本選擇策略",
            "團隊協作時構建一致",
        },
    }
    
    for _, scenario := range scenarios {
        fmt.Printf("場景: %s\n", scenario.title)
        fmt.Printf("情況: %s\n", scenario.situation)
        fmt.Printf("MVS 結果: %s\n", scenario.mvsResult)
        fmt.Printf("優勢: %s\n\n", scenario.benefit)
    }
    
    fmt.Println("使用 MVS 的命令:")
    fmt.Println("go mod graph    - 查看依賴圖") // 輸出: go mod graph    - 查看依賴圖
    fmt.Println("go mod why pkg  - 解釋為什麼需要某個依賴") // 輸出: go mod why pkg  - 解釋為什麼需要某個依賴
    fmt.Println("go list -m all  - 列出所有選中的模組版本") // 輸出: go list -m all  - 列出所有選中的模組版本
    
    fmt.Println("\nMVS 確保:")
    fmt.Println("✓ 構建結果可重現") // 輸出: ✓ 構建結果可重現
    fmt.Println("✓ 版本選擇可預測") // 輸出: ✓ 版本選擇可預測
    fmt.Println("✓ 升級路徑清晰") // 輸出: ✓ 升級路徑清晰
    fmt.Println("✓ 適合大型項目") // 輸出: ✓ 適合大型項目
}

### 更新到相容版本

相容版本更新是指在同一主版本內進行的升級，這些更新應該保持 API 相容性，不會破壞現有代碼。Go 的模組系統使相容版本更新變得安全和簡便。

**相容版本更新的類型**：
1. **補丁更新** (v1.2.3 → v1.2.4)：錯誤修復，完全向後相容
2. **次版本更新** (v1.2.3 → v1.3.0)：新功能，向後相容
3. **預發布到正式版** (v1.3.0-beta → v1.3.0)：穩定性提升

**更新策略**：
- **保守策略**：只更新補丁版本
- **平衡策略**：更新次版本，定期檢查
- **積極策略**：跟進最新版本，及時測試

**更新命令**：
- `go get -u=patch`：只更新補丁版本
- `go get -u`：更新到最新相容版本
- `go get package@upgrade`：升級特定包

**最佳實踐**：
- 定期檢查依賴更新
- 閱讀更新日誌
- 在測試環境驗證
- 監控性能和穩定性

In [ ]:
/* 相容版本更新流程 */
package main

import (
    "fmt"
    "time"
)

func main() {
    fmt.Println("相容版本更新實務指南")
    
    // 1. 檢查可更新的依賴
    fmt.Println("\n1. 檢查可更新的依賴:")
    checkUpdatableDependencies()
    
    // 2. 更新策略選擇
    fmt.Println("\n2. 更新策略選擇:")
    chooseUpdateStrategy()
    
    // 3. 執行更新流程
    fmt.Println("\n3. 執行更新流程:")
    executeUpdateProcess()
    
    // 4. 更新後驗證
    fmt.Println("\n4. 更新後驗證:")
    verifyAfterUpdate()
    
    // 5. 回滾策略
    fmt.Println("\n5. 回滾策略:")
    rollbackStrategy()
}

func checkUpdatableDependencies() {
    fmt.Println("檢查依賴更新的命令:")
    
    commands := []struct {
        command     string
        description string
        output      string
    }{
        {
            "go list -u -m all",
            "列出所有模組及可用更新",
            "github.com/gin-gonic/gin v1.9.0 [v1.9.1]",
        },
        {
            "go mod outdated",
            "顯示過時的依賴（需要工具）",
            "第三方工具，需要安裝",
        },
        {
            "go list -m -versions package",
            "查看包的所有可用版本",
            "v1.8.0 v1.8.1 v1.9.0 v1.9.1",
        },
    }
    
    for _, cmd := range commands {
        fmt.Printf("命令: %s\n", cmd.command)
        fmt.Printf("用途: %s\n", cmd.description)
        fmt.Printf("示例輸出: %s\n\n", cmd.output)
    }
    
    fmt.Println("依賴分析:")
    dependencies := []struct {
        name           string
        current        string
        available      string
        updateType     string
        recommendation string
    }{
        {
            "github.com/gin-gonic/gin",
            "v1.9.0",
            "v1.9.1",
            "PATCH",
            "建議更新（錯誤修復）",
        },
        {
            "github.com/sirupsen/logrus",
            "v1.8.1",
            "v1.9.3",
            "MINOR",
            "可以更新（新功能）",
        },
        {
            "github.com/stretchr/testify",
            "v1.7.0",
            "v1.8.4",
            "MINOR",
            "建議更新（測試改進）",
        },
    }
    
    fmt.Printf("%-30s %-10s %-10s %-8s %s\n", "套件", "當前版本", "可用版本", "類型", "建議")
    fmt.Println(strings.Repeat("-", 80))
    
    for _, dep := range dependencies {
        fmt.Printf("%-30s %-10s %-10s %-8s %s\n",
            dep.name, dep.current, dep.available, dep.updateType, dep.recommendation)
    }
}

func chooseUpdateStrategy() {
    strategies := []struct {
        strategy    string
        command     string
        riskLevel   string
        useCase     string
    }{
        {
            "保守更新（僅補丁）",
            "go get -u=patch",
            "低風險",
            "生產環境，穩定性優先",
        },
        {
            "標準更新（相容版本）",
            "go get -u",
            "中風險",
            "開發環境，平衡新功能和穩定性",
        },
        {
            "選擇性更新",
            "go get package@version",
            "可控風險",
            "針對性更新特定依賴",
        },
        {
            "測試版本",
            "go get package@latest",
            "高風險",
            "實驗環境，體驗最新功能",
        },
    }
    
    for _, strategy := range strategies {
        fmt.Printf("策略: %s\n", strategy.strategy)
        fmt.Printf("命令: %s\n", strategy.command)
        fmt.Printf("風險: %s\n", strategy.riskLevel)
        fmt.Printf("適用: %s\n\n", strategy.useCase)
    }
    
    fmt.Println("選擇策略的考量因素:")
    fmt.Println("1. 項目階段（開發/測試/生產）") // 輸出: 1. 項目階段（開發/測試/生產）
    fmt.Println("2. 更新緊急程度（安全修復/新功能）") // 輸出: 2. 更新緊急程度（安全修復/新功能）
    fmt.Println("3. 測試覆蓋率（高覆蓋率可承受更多風險）") // 輸出: 3. 測試覆蓋率（高覆蓋率可承受更多風險）
    fmt.Println("4. 回滾能力（是否容易回退）") // 輸出: 4. 回滾能力（是否容易回退）
}

func executeUpdateProcess() {
    fmt.Println("標準更新流程:")
    
    steps := []struct {
        step        string
        command     string
        description string
    }{
        {
            "1. 備份當前狀態",
            "git commit -am 'Pre-update snapshot'",
            "保存當前工作狀態",
        },
        {
            "2. 檢查當前依賴",
            "go list -m all > deps_before.txt",
            "記錄更新前的依賴版本",
        },
        {
            "3. 執行更新",
            "go get -u",
            "更新所有相容依賴",
        },
        {
            "4. 清理模組",
            "go mod tidy",
            "移除未使用的依賴",
        },
        {
            "5. 記錄變更",
            "go list -m all > deps_after.txt",
            "記錄更新後的依賴版本",
        },
        {
            "6. 比較差異",
            "diff deps_before.txt deps_after.txt",
            "查看具體變更",
        },
    }
    
    for _, step := range steps {
        fmt.Printf("%s\n", step.step)
        fmt.Printf("   命令: %s\n", step.command)
        fmt.Printf("   說明: %s\n\n", step.description)
    }
    
    fmt.Println("選擇性更新示例:")
    fmt.Println("# 只更新 Gin 框架到最新補丁版本")
    fmt.Println("go get github.com/gin-gonic/gin@patch") // 輸出: go get github.com/gin-gonic/gin@patch
    
    fmt.Println("# 更新到特定版本")
    fmt.Println("go get github.com/gin-gonic/gin@v1.9.1") // 輸出: go get github.com/gin-gonic/gin@v1.9.1
    
    fmt.Println("# 更新多個特定包")
    fmt.Println("go get github.com/gin-gonic/gin@upgrade github.com/sirupsen/logrus@latest")
    // 輸出: go get github.com/gin-gonic/gin@upgrade github.com/sirupsen/logrus@latest
}

func verifyAfterUpdate() {
    fmt.Println("更新後驗證清單:")
    
    verificationSteps := []struct {
        check       string
        command     string
        expectation string
    }{
        {
            "編譯檢查",
            "go build ./...",
            "無編譯錯誤",
        },
        {
            "測試執行",
            "go test ./...",
            "所有測試通過",
        },
        {
            "依賴驗證",
            "go mod verify",
            "所有依賴驗證通過",
        },
        {
            "漏洞掃描",
            "go list -json -deps ./... | nancy sleuth",
            "無已知漏洞",
        },
        {
            "性能測試",
            "go test -bench=. ./...",
            "性能無明顯下降",
        },
        {
            "整合測試",
            "運行端到端測試",
            "功能正常運作",
        },
    }
    
    for _, step := range verificationSteps {
        fmt.Printf("☐ %s\n", step.check)
        fmt.Printf("   命令: %s\n", step.command)
        fmt.Printf("   預期: %s\n\n", step.expectation)
    }
    
    fmt.Println("驗證失敗的常見問題:")
    issues := []struct {
        problem  string
        solution string
    }{
        {
            "編譯錯誤",
            "檢查 API 變更，更新代碼",
        },
        {
            "測試失敗",
            "檢查測試假設，更新測試代碼",
        },
        {
            "性能下降",
            "分析性能變化，考慮配置調整",
        },
        {
            "功能異常",
            "檢查配置文件和環境變數",
        },
    }
    
    for _, issue := range issues {
        fmt.Printf("問題: %s → 解決: %s\n", issue.problem, issue.solution)
    }
}

func rollbackStrategy() {
    fmt.Println("回滾策略:")
    
    fmt.Println("1. Git 回滾（推薦）:")
    fmt.Println("   git reset --hard HEAD~1") // 輸出:    git reset --hard HEAD~1
    fmt.Println("   恢復到更新前的 commit")  // 輸出:    恢復到更新前的 commit
    
    fmt.Println("\n2. 手動降級:")
    fmt.Println("   go get package@v1.8.1")   // 輸出:    go get package@v1.8.1
    fmt.Println("   go mod tidy")             // 輸出:    go mod tidy
    fmt.Println("   降級特定依賴版本")         // 輸出:    降級特定依賴版本
    
    fmt.Println("\n3. 使用備份的 go.mod:")
    fmt.Println("   cp go.mod.backup go.mod") // 輸出:    cp go.mod.backup go.mod
    fmt.Println("   go mod download")         // 輸出:    go mod download
    fmt.Println("   恢復依賴版本")             // 輸出:    恢復依賴版本
    
    fmt.Println("\n回滾最佳實踐:")
    fmt.Println("- 更新前創建分支或標籤") // 輸出: - 更新前創建分支或標籤
    fmt.Println("- 記錄更新的具體原因")   // 輸出: - 記錄更新的具體原因
    fmt.Println("- 建立回滾檢驗流程")     // 輸出: - 建立回滾檢驗流程
    fmt.Println("- 通知團隊成員變更")     // 輸出: - 通知團隊成員變更
}

### 更新至不相容的版本

主版本更新（如從 v1.x.x 到 v2.x.x）通常包含不相容的變更，需要特別小心處理。Go 的模組系統將不同主版本視為不同的模組，這需要特殊的處理方式。

**主版本更新的特點**：
1. **模組路徑變更**：v2+ 需要在路徑中包含版本號
2. **API 不相容**：可能有重大的 API 變更
3. **手動遷移**：需要手動更新 import 語句
4. **並存支持**：新舊版本可以同時存在

**更新流程**：
1. **評估影響**：分析 API 變更和重大更新
2. **更新路徑**：修改 import 語句包含版本號
3. **代碼適配**：根據新 API 調整代碼
4. **測試驗證**：全面測試確保功能正常

**Go 的版本管理規則**：
- v0.x.x：不穩定版本，任何更新都可能不相容
- v1.x.x：穩定版本，保持向後相容
- v2+.x.x：主版本更新，允許不相容變更

**處理策略**：
- 漸進式遷移：保持新舊版本並存
- 一次性遷移：統一升級到新版本
- 分階段遷移：按模組逐步升級

In [ ]:
/* 主版本升級處理 */
package main

import (
    "fmt"
    "strings"
)

func main() {
    fmt.Println("主版本升級處理指南")
    
    // 1. 評估升級影響
    fmt.Println("\n1. 評估升級影響:")
    assessUpgradeImpact()
    
    // 2. 升級路徑示例
    fmt.Println("\n2. 升級路徑示例:")
    showUpgradePath()
    
    // 3. 代碼遷移策略
    fmt.Println("\n3. 代碼遷移策略:")
    migrationStrategies()
    
    // 4. 並存方案
    fmt.Println("\n4. 並存方案:")
    coexistenceApproach()
    
    // 5. 常見問題處理
    fmt.Println("\n5. 常見問題處理:")
    handleCommonIssues()
}

func assessUpgradeImpact() {
    fmt.Println("升級前的影響評估:")
    
    assessmentSteps := []struct {
        step        string
        action      string
        tools       string
        importance  string
    }{
        {
            "查看更新日誌",
            "閱讀 CHANGELOG 和 BREAKING CHANGES",
            "GitHub Release Notes",
            "關鍵",
        },
        {
            "API 變更分析",
            "比較新舊版本的 API 差異",
            "go doc, 文檔比較",
            "關鍵",
        },
        {
            "依賴影響評估",
            "檢查其他依賴是否支持新版本",
            "go mod graph",
            "重要",
        },
        {
            "測試覆蓋率檢查",
            "確保有足夠的測試覆蓋",
            "go test -cover",
            "重要",
        },
        {
            "回退計劃制定",
            "準備升級失敗的回退方案",
            "Git branch/tag",
            "必須",
        },
    }
    
    for _, step := range assessmentSteps {
        fmt.Printf("%s:\n", step.step)
        fmt.Printf("   行動: %s\n", step.action)
        fmt.Printf("   工具: %s\n", step.tools)
        fmt.Printf("   重要性: %s\n\n", step.importance)
    }
    
    fmt.Println("風險評估矩陣:")
    riskFactors := []struct {
        factor     string
        lowRisk    string
        highRisk   string
    }{
        {
            "API 變更程度",
            "少量新增功能",
            "大量 API 重構",
        },
        {
            "測試覆蓋率",
            "> 80%",
            "< 50%",
        },
        {
            "項目規模",
            "小型項目",
            "大型分散式系統",
        },
        {
            "團隊經驗",
            "豐富升級經驗",
            "首次主版本升級",
        },
    }
    
    fmt.Printf("%-15s %-20s %s\n", "因素", "低風險", "高風險")
    fmt.Println(strings.Repeat("-", 60))
    
    for _, factor := range riskFactors {
        fmt.Printf("%-15s %-20s %s\n", factor.factor, factor.lowRisk, factor.highRisk)
    }
}

func showUpgradePath() {
    fmt.Println("典型升級路徑示例（GORM v1 → v2）:")
    
    fmt.Println("\n原始代碼 (v1):")
    v1Code := `import (
    "github.com/jinzhu/gorm"
    _ "github.com/jinzhu/gorm/dialects/postgres"
)

func main() {
    db, err := gorm.Open("postgres", dsn)
    if err != nil {
        panic(err)
    }
    defer db.Close()
    
    var user User
    db.First(&user, 1)
}`
    
    fmt.Println(v1Code)
    
    fmt.Println("\n升級後代碼 (v2):")
    v2Code := `import (
    "gorm.io/gorm"
    "gorm.io/driver/postgres"
)

func main() {
    db, err := gorm.Open(postgres.Open(dsn), &gorm.Config{})
    if err != nil {
        panic(err)
    }
    
    var user User
    result := db.First(&user, 1)
    if result.Error != nil {
        // 處理錯誤
    }
}`
    
    fmt.Println(v2Code)
    
    fmt.Println("\n主要變更點:")
    changes := []struct {
        aspect string
        v1     string
        v2     string
    }{
        {
            "模組路徑",
            "github.com/jinzhu/gorm",
            "gorm.io/gorm",
        },
        {
            "驅動匯入",
            "_ \"github.com/jinzhu/gorm/dialects/postgres\"",
            "\"gorm.io/driver/postgres\"",
        },
        {
            "連接方式",
            "gorm.Open(\"postgres\", dsn)",
            "gorm.Open(postgres.Open(dsn), &gorm.Config{})",
        },
        {
            "錯誤處理",
            "db.Error",
            "result.Error",
        },
    }
    
    for _, change := range changes {
        fmt.Printf("%s:\n", change.aspect)
        fmt.Printf("   v1: %s\n", change.v1)
        fmt.Printf("   v2: %s\n\n", change.v2)
    }
}

func migrationStrategies() {
    strategies := []struct {
        strategy    string
        approach    string
        pros        string
        cons        string
        bestFor     string
    }{
        {
            "Big Bang 遷移",
            "一次性全面升級",
            "簡潔，一次完成",
            "風險高，回退困難",
            "小型項目",
        },
        {
            "分階段遷移",
            "按模組逐步升級",
            "風險可控，漸進式",
            "過程較長，維護復雜",
            "中大型項目",
        },
        {
            "並行開發",
            "新舊版本同時存在",
            "零停機升級",
            "代碼重複，複雜度高",
            "關鍵生產系統",
        },
        {
            "特性分支",
            "在分支中完成升級",
            "不影響主線開發",
            "合併時可能有衝突",
            "團隊協作項目",
        },
    }
    
    for _, strategy := range strategies {
        fmt.Printf("策略: %s\n", strategy.strategy)
        fmt.Printf("方法: %s\n", strategy.approach)
        fmt.Printf("優點: %s\n", strategy.pros)
        fmt.Printf("缺點: %s\n", strategy.cons)
        fmt.Printf("適用: %s\n\n", strategy.bestFor)
    }
    
    fmt.Println("分階段遷移示例:")
    phases := []string{
        "階段 1: 升級核心工具模組",
        "階段 2: 升級數據訪問層",
        "階段 3: 升級業務邏輯層",
        "階段 4: 升級 API 層",
        "階段 5: 升級測試代碼",
    }
    
    for _, phase := range phases {
        fmt.Printf("  %s\n", phase)
    }
}

func coexistenceApproach() {
    fmt.Println("新舊版本並存方案:")
    
    fmt.Println("go.mod 中同時使用兩個版本:")
    coexistExample := `module myproject

go 1.19

require (
    github.com/old/package v1.5.2
    github.com/old/package/v2 v2.1.0
)

// 或者使用 replace
replace github.com/old/package/v2 => github.com/new/package v2.1.0`
    
    fmt.Println(coexistExample)
    
    fmt.Println("\n代碼中使用別名區分:")
    aliasExample := `import (
    oldpkg "github.com/old/package"
    newpkg "github.com/old/package/v2"
)

func migrate() {
    // 逐步將 oldpkg 的使用遷移到 newpkg
    oldClient := oldpkg.NewClient()
    newClient := newpkg.NewClient()
    
    // 可以比較結果，確保一致性
    oldResult := oldClient.DoSomething()
    newResult := newClient.DoSomething()
    
    // 驗證結果一致性
    if !compareResults(oldResult, newResult) {
        log.Warn("結果不一致，需要調查")
    }
}`
    
    fmt.Println(aliasExample)
    
    fmt.Println("\n並存的優勢:")
    fmt.Println("- 可以逐步遷移，降低風險") // 輸出: - 可以逐步遷移，降低風險
    fmt.Println("- 可以進行 A/B 測試") // 輸出: - 可以進行 A/B 測試
    fmt.Println("- 出問題時可以快速回退") // 輸出: - 出問題時可以快速回退
    fmt.Println("- 可以比較新舊版本的行為") // 輸出: - 可以比較新舊版本的行為
    
    fmt.Println("\n注意事項:")
    fmt.Println("- 會增加二進制檔案大小") // 輸出: - 會增加二進制檔案大小
    fmt.Println("- 需要維護兩套代碼路徑") // 輸出: - 需要維護兩套代碼路徑
    fmt.Println("- 可能有間接依賴衝突") // 輸出: - 可能有間接依賴衝突
}

func handleCommonIssues() {
    fmt.Println("主版本升級常見問題及解決方案:")
    
    issues := []struct {
        issue       string
        symptom     string
        solution    string
        prevention  string
    }{
        {
            "Import 路徑錯誤",
            "編譯錯誤: package not found",
            "更新 import 路徑包含版本號",
            "使用 IDE 的重構工具",
        },
        {
            "API 簽名變更",
            "編譯錯誤: function signature mismatch",
            "根據新 API 調整函數調用",
            "詳細閱讀遷移指南",
        },
        {
            "行為變更",
            "功能異常但無編譯錯誤",
            "檢查文檔，調整代碼邏輯",
            "充分的集成測試",
        },
        {
            "性能變化",
            "程序變慢或內存使用增加",
            "分析性能，調整配置",
            "性能基準測試",
        },
        {
            "間接依賴衝突",
            "依賴解析錯誤",
            "使用 replace 指令解決",
            "依賴圖分析",
        },
    }
    
    for _, issue := range issues {
        fmt.Printf("問題: %s\n", issue.issue)
        fmt.Printf("症狀: %s\n", issue.symptom)
        fmt.Printf("解決: %s\n", issue.solution)
        fmt.Printf("預防: %s\n\n", issue.prevention)
    }
    
    fmt.Println("升級成功的關鍵因素:")
    fmt.Println("1. 充分的準備和評估") // 輸出: 1. 充分的準備和評估
    fmt.Println("2. 完整的測試覆蓋") // 輸出: 2. 完整的測試覆蓋
    fmt.Println("3. 分階段的遷移計劃") // 輸出: 3. 分階段的遷移計劃
    fmt.Println("4. 及時的問題反饋和修復") // 輸出: 4. 及時的問題反饋和修復
    fmt.Println("5. 團隊的密切協作") // 輸出: 5. 團隊的密切協作
}

## vendoring

Vendoring 是將依賴項的副本存儲在項目內的 vendor/ 目錄中的做法。雖然 Go 模組系統減少了對 vendoring 的需求，但在某些情況下它仍然很有用。

**Vendoring 的用途**：
1. **離線構建**：不依賴網路的構建環境
2. **依賴穩定性**：避免上游依賴消失或變更
3. **企業環境**：滿足安全和合規要求
4. **構建速度**：減少下載時間

**Vendoring 的工作原理**：
- `go mod vendor` 命令創建 vendor/ 目錄
- 復制所有直接和間接依賴到 vendor/
- 構建時使用 `-mod=vendor` 標志
- 優先使用 vendor/ 中的代碼

**使用場景**：
- CI/CD 環境需要可重複構建
- 企業防火牆限制外部下載
- 需要修改第三方依賴代碼
- 部署到無網路環境

**最佳實踐**：
- vendor/ 目錄是否提交到版本控制取決於需求
- 定期更新 vendor 內容
- 保持 go.mod 和 vendor/ 的同步

In [ ]:
/* vendoring 機制的使用 */
package main

import (
    "fmt"
    "os"
    "path/filepath"
)

func main() {
    fmt.Println("Go Vendoring 完整指南")
    
    // 1. Vendoring 基本操作
    fmt.Println("\n1. Vendoring 基本操作:")
    basicVendoring()
    
    // 2. Vendoring 使用場景
    fmt.Println("\n2. Vendoring 使用場景:")
    vendoringUseCases()
    
    // 3. Vendor 目錄結構
    fmt.Println("\n3. Vendor 目錄結構:")
    vendorStructure()
    
    // 4. 企業環境實踐
    fmt.Println("\n4. 企業環境實踐:")
    enterprisePractices()
    
    // 5. Vendoring 最佳實踐
    fmt.Println("\n5. Vendoring 最佳實踐:")
    bestPractices()
}

func basicVendoring() {
    fmt.Println("基本 vendoring 操作:")
    
    operations := []struct {
        command     string
        description string
        example     string
    }{
        {
            "go mod vendor",
            "創建 vendor 目錄並複製依賴",
            "複製所有依賴到 ./vendor/",
        },
        {
            "go build -mod=vendor",
            "使用 vendor 中的依賴構建",
            "優先使用本地 vendor 依賴",
        },
        {
            "go test -mod=vendor",
            "使用 vendor 中的依賴測試",
            "確保測試使用相同依賴",
        },
        {
            "go mod tidy && go mod vendor",
            "更新並重建 vendor",
            "同步最新依賴到 vendor",
        },
    }
    
    for _, op := range operations {
        fmt.Printf("命令: %s\n", op.command)
        fmt.Printf("說明: %s\n", op.description)
        fmt.Printf("效果: %s\n\n", op.example)
    }
    
    fmt.Println("Vendoring 工作流程:")
    workflow := []string{
        "1. 初始化項目: go mod init myproject",
        "2. 添加依賴: go get github.com/gin-gonic/gin",
        "3. 創建 vendor: go mod vendor",
        "4. 構建項目: go build -mod=vendor",
        "5. 提交代碼: git add . && git commit",
    }
    
    for _, step := range workflow {
        fmt.Printf("   %s\n", step)
    }
}

func vendoringUseCases() {
    useCases := []struct {
        scenario    string
        problem     string
        solution    string
        benefits    string
    }{
        {
            "CI/CD 環境",
            "構建環境無法訪問外部網路",
            "使用 vendor 目錄提供離線依賴",
            "構建速度快，不依賴網路",
        },
        {
            "企業安全要求",
            "安全政策禁止下載外部依賴",
            "預先審核並 vendor 所有依賴",
            "滿足合規要求，控制風險",
        },
        {
            "依賴穩定性",
            "擔心上游依賴被刪除或修改",
            "vendor 確保依賴永遠可用",
            "構建結果可重現",
        },
        {
            "本地修改依賴",
            "需要修改第三方庫的行為",
            "vendor 後直接修改源代碼",
            "快速修復和定制",
        },
        {
            "離線開發",
            "開發環境網路不穩定",
            "vendor 支持完全離線開發",
            "提高開發效率",
        },
    }
    
    for _, useCase := range useCases {
        fmt.Printf("場景: %s\n", useCase.scenario)
        fmt.Printf("問題: %s\n", useCase.problem)
        fmt.Printf("方案: %s\n", useCase.solution)
        fmt.Printf("優勢: %s\n\n", useCase.benefits)
    }
}

func vendorStructure() {
    fmt.Println("Vendor 目錄結構示例:")
    
    structure := `myproject/
├── go.mod
├── go.sum
├── main.go
└── vendor/
    ├── github.com/
    │   ├── gin-gonic/
    │   │   └── gin/
    │   │       ├── gin.go
    │   │       ├── context.go
    │   │       └── ...
    │   └── sirupsen/
    │       └── logrus/
    │           ├── logrus.go
    │           └── ...
    ├── golang.org/
    │   └── x/
    │       └── sys/
    │           └── ...
    └── modules.txt`
    
    fmt.Println(structure)
    
    fmt.Println("關鍵文件說明:")
    files := []struct {
        file        string
        description string
    }{
        {
            "vendor/modules.txt",
            "記錄了 vendor 中包含的模組和版本",
        },
        {
            "vendor/<package>/",
            "包含第三方包的完整源代碼",
        },
        {
            "go.mod",
            "仍然需要，記錄依賴關係",
        },
        {
            "go.sum",
            "仍然需要，用於驗證依賴完整性",
        },
    }
    
    for _, file := range files {
        fmt.Printf("%-20s - %s\n", file.file, file.description)
    }
    
    fmt.Println("\nmodules.txt 文件示例:")
    modulesExample := `# github.com/gin-gonic/gin v1.9.1
## explicit; go 1.19
github.com/gin-gonic/gin
github.com/gin-gonic/gin/binding
github.com/gin-gonic/gin/render
# github.com/sirupsen/logrus v1.9.3
## explicit
github.com/sirupsen/logrus`
    
    fmt.Println(modulesExample)
}

func enterprisePractices() {
    fmt.Println("企業環境 Vendoring 實踐:")
    
    practices := []struct {
        practice    string
        description string
        example     string
    }{
        {
            "依賴審核流程",
            "建立依賴審核和批准流程",
            "安全團隊審核所有新依賴",
        },
        {
            "私有依賴代理",
            "搭建內部依賴代理服務",
            "所有依賴通過內部代理下載",
        },
        {
            "定期安全掃描",
            "對 vendor 目錄進行安全掃描",
            "使用工具檢測已知漏洞",
        },
        {
            "版本控制策略",
            "決定是否將 vendor 提交到 Git",
            "大型項目可能選擇不提交",
        },
        {
            "自動化更新",
            "建立依賴更新的自動化流程",
            "定期檢查和更新依賴版本",
        },
    }
    
    for _, practice := range practices {
        fmt.Printf("%s:\n", practice.practice)
        fmt.Printf("   描述: %s\n", practice.description)
        fmt.Printf("   示例: %s\n\n", practice.example)
    }
    
    fmt.Println("企業級 vendoring 腳本示例:")
    scriptExample := `#!/bin/bash
# enterprise-vendor.sh

set -e

echo "開始企業級 vendoring 流程..."

# 1. 清理舊的 vendor
rm -rf vendor/

# 2. 更新依賴
go mod tidy

# 3. 下載依賴
go mod download

# 4. 安全掃描
echo "執行安全掃描..."
nancy sleuth --loud < go.sum || exit 1

# 5. 創建 vendor
go mod vendor

# 6. 驗證 vendor
go build -mod=vendor ./... || exit 1
go test -mod=vendor ./... || exit 1

# 7. 生成依賴報告
echo "生成依賴報告..."
go list -m -json all > dependencies-report.json

echo "Vendoring 完成！"`
    
    fmt.Println(scriptExample)
}

func bestPractices() {
    fmt.Println("Vendoring 最佳實踐:")
    
    practices := []struct {
        category string
        rules    []string
    }{
        {
            "目錄管理",
            []string{
                "定期清理和重建 vendor 目錄",
                "保持 go.mod、go.sum 和 vendor 的同步",
                "不要手動修改 vendor 中的代碼",
                "考慮 vendor 目錄的磁盤空間占用",
            },
        },
        {
            "版本控制",
            []string{
                "小型項目可以提交 vendor 目錄",
                "大型項目考慮使用 .gitignore 排除",
                "在 CI 環境中重建 vendor",
                "提供清晰的構建文檔",
            },
        },
        {
            "安全考量",
            []string{
                "定期掃描 vendor 中的安全漏洞",
                "建立依賴更新的流程",
                "記錄依賴使用的原因和風險",
                "監控上游依賴的安全公告",
            },
        },
        {
            "性能優化",
            []string{
                "使用 go mod download 預下載",
                "在 CI 中快取 vendor 目錄",
                "考慮使用模組代理加速",
                "監控構建時間變化",
            },
        },
    }
    
    for _, category := range practices {
        fmt.Printf("%s:\n", category.category)
        for _, rule := range category.rules {
            fmt.Printf("   - %s\n", rule)
        }
        fmt.Println()
    }
    
    fmt.Println("Vendoring vs 模組系統對比:")
    fmt.Printf("%-15s %-25s %s\n", "方面", "模組系統", "Vendoring")
    fmt.Println(strings.Repeat("-", 65))
    
    comparisons := []struct {
        aspect string
        module string
        vendor string
    }{
        {"網路依賴", "需要網路下載", "本地副本，離線可用"},
        {"磁盤空間", "共享快取，節省空間", "每項目獨立，占用更多"},
        {"構建速度", "首次較慢", "始終快速"},
        {"更新管理", "自動解決版本", "手動管理"},
        {"安全性", "依賴代理驗證", "本地控制"},
        {"合規性", "可能有限制", "容易滿足要求"},
    }
    
    for _, comp := range comparisons {
        fmt.Printf("%-15s %-25s %s\n", comp.aspect, comp.module, comp.vendor)
    }
    
    fmt.Println("\n選擇指導:")
    fmt.Println("使用模組系統：") // 輸出: 使用模組系統：
    fmt.Println("- 一般開發環境") // 輸出: - 一般開發環境
    fmt.Println("- 網路穩定的環境") // 輸出: - 網路穩定的環境
    fmt.Println("- 磁盤空間有限") // 輸出: - 磁盤空間有限
    
    fmt.Println("\n使用 Vendoring：") // 輸出: 使用 Vendoring：
    fmt.Println("- 企業環境和合規要求") // 輸出: - 企業環境和合規要求
    fmt.Println("- 離線或限制網路環境") // 輸出: - 離線或限制網路環境
    fmt.Println("- 需要修改第三方依賴") // 輸出: - 需要修改第三方依賴
}